In [0]:
!pip install xlrd==1.2.0
!pip install openpyxl
!pip install geopandas
!pip install geopy

Requirement already satisfied: xlrd==1.2.0 in /databricks/python3/lib/python3.8/site-packages (1.2.0)
 WARNING: You are using pip version 21.0.1; however, version 22.0.4 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Requirement already satisfied: openpyxl in /databricks/python3/lib/python3.8/site-packages (3.0.9)
Requirement already satisfied: et-xmlfile in /databricks/python3/lib/python3.8/site-packages (from openpyxl) (1.1.0)
 WARNING: You are using pip version 21.0.1; however, version 22.0.4 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Requirement already satisfied: geopandas in /databricks/python3/lib/python3.8/site-packages (0.10.2)
Requirement already satisfied: fiona>=1.8 in /databricks/python3/lib/python3.8/site-packages (from geopandas) (1.8.21)
Requirement already satisfied: shapely>=1.6 in /databricks/python3/lib/python3.8/site-packages (from geopandas) (1.8.1.post1)
Requirement already satisfied: pyproj>=2.2.0 in /databricks/python3/lib/python3.8/site-packages (from geopandas) (3.3.0)
Requirement already satisfied: pandas>=0.25.0 in /databricks/python3/lib/python3.8/site-packages (from geopandas) (1.2.4)
Requirement already satisfied: six>=1.7 in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (1.15.0)
Requirement already satisfied: setuptools in /usr/local/lib/python3.8/dist-packages (from fiona>=1.8->geopandas) (52.0.0)
Requirement already satisfied: click>=4.0 in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (8.0.4)
Requirement already satisfied: certifi in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (2020.12.5)
Requirement already satisfied: munch in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (2.5.0)
Requirement already satisfied: cligj>=0.5 in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (0.7.2)
Requirement already satisfied: click-plugins>=1.0 in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (1.1.1)
Requirement already satisfied: attrs>=17 in /databricks/python3/lib/python3.8/site-packages (from fiona>=1.8->geopandas) (20.3.0)
Requirement already satisfied: pytz>=2017.3 in /databricks/python3/lib/python3.8/site-packages (from pandas>=0.25.0->geopandas) (2020.5)
Requirement already satisfied: numpy>=1.16.5 in /databricks/python3/lib/python3.8/site-packages (from pandas>=0.25.0->geopandas) (1.20.1)
Requirement already satisfied: python-dateutil>=2.7.3 in /databricks/python3/lib/python3.8/site-packages (from pandas>=0.25.0->geopandas) (2.8.1)
 WARNING: You are using pip version 21.0.1; however, version 22.0.4 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Requirement already satisfied: geopy in /databricks/python3/lib/python3.8/site-packages (2.2.0)
Requirement already satisfied: geographiclib<2,>=1.49 in /databricks/python3/lib/python3.8/site-packages (from geopy) (1.52)
 WARNING: You are using pip version 21.0.1; however, version 22.0.4 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [0]:
import pandas as pd
import numpy as np
import datetime
from datetime import date, timedelta
import os
import warnings
import xlrd
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
import requests
import json
import geopandas
from geopy.geocoders import Photon
from numpy import cos, sin, arcsin, sqrt
from math import radians
import scipy.stats as stats
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [0]:
# in_scop_mat='enter file path to 171Material_inScope.csv'
# #Reading in-scope material file
# in_scope_material=pd.read_csv(in_scop_mat)

# ves_flat_file='enter file path to VES_data_final_08_12_v1.csv'
# #Reading the combined files of missed PO and all POs data concatenated in excel
# ves_final=pd.read_csv(ves_flat_file)

# ves_old_flat_file = 'enter file path to VES_data_final_06_12.csv'
# #Reading VES old data to get PO Create Date for 2020 POs
# VES_old_data = pd.read_csv(ves_old_flat_file)

# PO_Create_Date_file = 'enter file path to PO_Creation_Date.xlsx'
# #Reading PO_Creation_Date data to extract PO Create Date for 2021 POs
# PO_create_date = pd.read_excel(PO_Create_Date_file, skiprows=3)

# plant_city_state_file = 'enter file path to Plant_lat_long.csv'
# #reading file containing Plant City ans State
# plant_city_state = pd.read_csv(plant_city_state_file)

# #sheet names ['5004 SOLON', '5878 JONESBORO', '5955 GAFFNEY', '5243 MOSS LANDING', '5959 SPRINGVILLE']
# spend_analy='enter file path to Spend_Analysis_Invoice_Based_with_Vendor_Site_2020throughYTD_v3__1___1_.xlsx'
# #sheet names ['5259 LIttle Chute Pack 2020', '5258 MEDFORD_Pack', '5258 MEDFORD_Raw','5446 MT STERLING Raw', '5446 MT STERLING Pack']
# spend_analy1='enter file path to Spend_Analysis_Invoice_Based_with_Vendor_Site_2020throughYTD_v3__2___1_.xlsx'
# #sheet names Report2 plant 5259
# spend_analy2='enter file path to 5259_Raw__1_.xlsx'

# smi_file='enter file path to SMI_data_bricks_raw_data_csv.xlsx'
# #reading the SMI file
# smi_1=pd.read_excel(smi_file)

# fin_recds='enter file path to Risk_Support_raw_data.xlsx'
# #reading financial records
# fin=pd.read_excel(fin_recds)

# #reading inventory file
# inventory_flat_file="enter file path to inventory1.csv"
# #reading MB51 consumption file
# MB51_flat_file = 'enter file path to /ROH_Frozen_Plant___ZPCK___2020__1_.csv'
# MB51_data=pd.read_csv(MB51_flat_file)

# #Reading QIR data to get missing VS locations from VES
# QIR_file = 'enter file path to QIR_updated_copy.csv'
# QIR_updated_data = pd.read_csv(QIR_file)

# #Reading Incoterms data
# incoterms_file = 'enter file path to incoterms_file_25_12.csv'
# incoterms_data = pd.read_csv(incoterms_file)

In [0]:
in_scop_mat='enter file path to 171Material_inScope.csv'
#Reading in-scope material file
in_scope_material=pd.read_csv(in_scop_mat)

ves_flat_file='enter file path to Vendor_Evaluation_Scorecard_ALL___03082022.xlsx'
#Reading the combined files of missed PO and all POs data concatenated in excel
ves_1=pd.read_excel(ves_flat_file,skiprows=[0,1,2,3,4],header=[1])

ves_flat_files='enter file path to Vendor_Evaluation_Scorecard_FAILS___03082022.xlsx'
#Reading the combined files of missed PO and all POs data concatenated in excel
ves_2=pd.read_excel(ves_flat_files)

# ves_old_flat_file = 'enter file path to Vendor_Evaluation_Scorecard_FAILS___03082022.xlsx'
#Reading VES old data to get PO Create Date for 2020 POs
# VES_old_data = pd.read_excel(ves_old_flat_file)

PO_Create_Date_file = 'enter file path to PO_Creation_Date_03082022.xlsx'
#Reading PO_Creation_Date data to extract PO Create Date for 2021 POs
PO_create_date = pd.read_excel(PO_Create_Date_file,usecols=[1,2,3,4,5], skiprows=3)

plant_city_state_file = 'enter file path to Plant_lat_long.csv'
#reading file containing Plant City ans State
plant_city_state = pd.read_csv(plant_city_state_file)

#sheet names ['5004 SOLON', '5878 JONESBORO', '5955 GAFFNEY', '5243 MOSS LANDING', '5959 SPRINGVILLE']
spend_analy='enter file path to Spend_Analysis_16_03_cleaned.xlsx'

smi_file='enter file path to SMI_data_bricks_raw_data_csv.xlsx'
#reading the SMI file
smi_1=pd.read_excel(smi_file)

fin_recds='enter file path to Risk_Support_raw_data.xlsx'
#reading financial records
fin=pd.read_excel(fin_recds)

#reading inventory file
inventory_flat_file="enter file path to dec2021_mar2022.csv"
#reading MB51 consumption file
MB51_flat_file = 'enter file path to ROH_Frozen_Plant___ZPCK___2020__1_.csv'
MB51_data=pd.read_csv(MB51_flat_file)

#Reading QIR data to get missing VS locations from VES
QIR_file = 'enter file path to QIR_updated_copy.csv'
QIR_updated_data = pd.read_csv(QIR_file)

#Reading Incoterms data
incoterms_file = 'enter file path to incoterms_file_25_12.csv'
incoterms_data = pd.read_csv(incoterms_file)

In [0]:
#spend_analy1='enter file path to Spend_Analysis_16_03_cleaned.xlsx'
spend_analyd=pd.read_csv('enter file path to Spend_Analysis_16_03_cleaned.xlsx')
spend_analyd.head(1)

Out[48]:

,Unnamed: 0,Spend Group,Plant - Key,Plant - Text,Material - Key,Material - Text,Vendor - Key,Vendor - Text,Supplying Vendor - Key,Supplying Vendor - Text,PO Number - Key,PO Item Number,GR Vendor-Site - Key,GR Vendor-Site - CAM: City District (Key),GR Vendor-Site - Region (Key),Supplying Vendor - Country (Key),Batch Vendor-Site - CAM: City District (Key),Batch Vendor-Site - Country (Key),Batch Vendor-Site - Key,Batch Vendor-Site - Postal Code (Key),Batch Vendor-Site - Region (Key),Contract Reference Item Number,Contract Reference Number - Key,GR Mat Doc,GR Vendor-Site,GR Vendor-Site - Country (Key),GR Vendor-Site - Postal Code (Key),Material Document,Posting date,Supplying Vendor - Postal Code (Key),Supplying Vendor - CAM: City District (Key),Supplying Vendor - Location (Key),Supplying Vendor - Region (Key),PO Item Creation Date
0,0,Packaging,5004.0,US PL SOLON,22013995.0,Case Corr STFR Gnrc 12xBoil In Bag Pouch,100364660.0,International Paper Company,100364660,International Paper Company,4.567373e+09,10.0,100364660,shelby,tn,us,shelby,us,100364660,02852-7520,tn,1,A9APRR3103/MA076231,8672392559,Not assigned,us,#,5.335890e+09,2020-03-18,02852-7520,shelby,shelby,tn,2021-09-02


In [0]:
incoterms_files = 'enter file path to Planned_Delivery_Times_03082022.xlsx'
incoterms_datas = pd.read_excel(incoterms_files, skiprows=[0,1,2],usecols=[1,2,3],header=[0])
incoterms_datas.head(1)

Out[49]:

,Plant - Key,Material - Key,Mat. Plant View - Planned deliv. time (Key)
0,5004,22000021,0.0


In [0]:
PO_data = 'enter file path to ALL_POs__Nov2021_YTD.xlsx'
PO_line_item = pd.read_excel(PO_data, skiprows=[0,1,2,3,4,5],header=[0])
PO_line_item.rename(columns={'Unnamed: 3':'Plant Name','Unnamed: 5':'Material Description','Unnamed: 7':'VM Name','Unnamed: 9':'VS Name','$':'Invoice Amount (rep.curr.monthly rate)','$.1':'Invoice Amount','Unnamed: 14':'Invoice Qty','Unnamed: 15':'G.R. Delv. Qty','Plant':'Plant ID','Material':'Material No.','PO Number':'Purchase Order','PO Item Number':'PO Item'},inplace=True)
print(PO_line_item.shape)
PO_line_item.dropna(inplace=True)
print(PO_line_item.shape)
PO_line_item.head(1)

(24144, 16)
(24143, 16)
Out[50]:

,Spend Group,Cal. year / month,Plant ID,Plant Name,Material No.,Material Description,Vendor,VM Name,Supplying Vendor,VS Name,Purchase Order,PO Item,Invoice Amount (rep.curr.monthly rate),Invoice Amount,Invoice Qty,G.R. Delv. Qty
0,Packaging,11/2021,5004.0,US PL SOLON,22005956.0,Case Corr Tray STOUFFER'S Custom 6x36oz,100364660.0,International Paper Company,100364660.0,International Paper Company,4.567385e+09,10.0,2739.28,2739.28,20610.0,0.0


In [0]:
PO_line_cut=PO_line_item[['Plant ID','Material No.','Vendor','Purchase Order','PO Item']].copy()
print(PO_line_cut.shape)
PO_line_cut.drop_duplicates(inplace=True)
print(PO_line_cut.shape)
for i in PO_line_cut.columns:
  PO_line_cut[i]=PO_line_cut[i].astype(int)
PO_line_cut=PO_line_cut[PO_line_cut['Material No.'].isin(in_scope_material['Material'])]
print(PO_line_cut.shape)
PO_line_cut.head(1)

(24143, 5)
(20980, 5)
(11102, 5)
Out[51]:

,Plant ID,Material No.,Vendor,Purchase Order,PO Item
8,5004,22006408,100853829,4567427644,10


In [0]:
ves_1.rename(columns={'%':'On-Time Delivery(%)','%.1':'Quantity Reliability(%)','%.2':'Initial Quality(%)','%.3':'Total Quality(%)','%.4':'Supplier Service(%)','Unnamed: 27':'Purchase Order Scheduled Qty','Unnamed: 28':'Delivered Quantity','Unnamed: 1':'Plant Name','Unnamed: 3':'VM Name','Unnamed: 5':'VS Name','Unnamed: 8':'Material Group','Unnamed: 10':'Material Description','Unnamed: 12':'MRP','Plant':'Plant ID','Supplying Vendor':'Supplier No.','Material Group (master data)':'Material_group_type','Material':'Material No.'},inplace=True)
print(ves_1.shape)
ves_1.drop(index=ves_1[ves_1['First Statistical delivery Date']=='#'].index,axis=0,inplace=True)
print(ves_1.shape)
ves_1.drop(index=ves_1[ves_1['First GR Date']=='#'].index,axis=0,inplace=True)
ves_1['First GR Date']=pd.to_datetime(ves_1['First GR Date']).dt.date
ves_1['Last GR Date']=pd.to_datetime(ves_1['Last GR Date']).dt.date
ves_1['First Statistical delivery Date']=pd.to_datetime(ves_1['First Statistical delivery Date']).dt.date
ves_1['Last Statistical delivery Date']=pd.to_datetime(ves_1['Last Statistical delivery Date']).dt.date
ves_1.drop(columns=['MRP','Supplier Service(%)'],inplace=True)
print(ves_1.shape)
ves_1=ves_1[ves_1['Material No.'].isin(in_scope_material['Material'])]
print(ves_1.shape)
ves_11=ves_1.merge(PO_line_cut,on=['Plant ID','Material No.','Vendor','Purchase Order'],how='inner')
ves_11['Missed_PO']='No'
print(ves_11.shape)
ves_11.head(1)

(37208, 29)
(35968, 29)
(35967, 27)
(9923, 27)
(9993, 29)
Out[52]:

,Plant ID,Plant Name,Vendor,VM Name,Supplier No.,VS Name,Purchase Order,Material_group_type,Material Group,Material No.,Material Description,MRP controller,Manual Override (Y/N),First GR Date,Last GR Date,First Statistical delivery Date,Last Statistical delivery Date,Under Delievry Tolerance(%),Over Delivery Tolerance(%),Early Delivery Tolerance(Days),Late Delievry Tolerance(Days),On-Time Delivery(%),Quantity Reliability(%),Initial Quality(%),Total Quality(%),Purchase Order Scheduled Qty,Delivered Quantity,PO Item,Missed_PO
0,5004,US PL SOLON,100037407,Nestrade S.A.,100099158,Leprino Foods Dairy Products Co,4567777068,R01,Dairy Products,22002012,Cheese Mozz PrtSk Pmsn Flv Shd IQF,303,N,2021-11-19,2021-11-19,2021-11-19,2021-11-19,5,5,2,1,100,100.0,100,100,19800.0,19800.0,10,No


In [0]:
ves_11[ves_11['First GR Date']=='#'].shape,ves_11[ves_11['Last GR Date']=='#'].shape,ves_11[ves_11['First Statistical delivery Date']=='#'].shape,ves_11[ves_11['Last Statistical delivery Date']=='#'].shape

Out[53]: ((0, 29), (0, 29), (0, 29), (0, 29))

In [0]:
ves_cut=ves_1[['Plant ID','Material No.','Supplier No.','Vendor','VM Name']].copy()
ves_cut.drop_duplicates(inplace=True)
print(ves_cut.shape)
ves_cut.head()

(453, 5)
Out[54]:

,Plant ID,Material No.,Supplier No.,Vendor,VM Name
8,5004,22002012,100099158,100037407,Nestrade S.A.
19,5004,22000213,100364181,100037407,Nestrade S.A.
22,5004,43100075,100364181,100037407,Nestrade S.A.
30,5004,22002229,100364181,100037407,Nestrade S.A.
44,5004,22002410,100364965,100037407,Nestrade S.A.


In [0]:
print(ves_2.shape)
ves_2.dropna(inplace=True)
ves_2.rename(columns={'Plant':'Plant ID','Supplying Vendor':'VS Name','Material':'Material Description','Planner':'MRP controller','Manual Over-ride (Y/N)':'Manual Override (Y/N)','Last Stat. Delivery Date':'Last Statistical delivery Date','Under Delivery Tolerance(%)':'Under Delievry Tolerance(%)','Late Delivery Tolerance(Days)':'Late Delievry Tolerance(Days)','Scheduled Qty':'Purchase Order Scheduled Qty','On-Time Delivery (%)':'On-Time Delivery(%)','Quantity Reliability (%)':'Quantity Reliability(%)','Initial Quality (%)':'Initial Quality(%)','Total Quality (%)':'Total Quality(%)','Material Group':'Material_group_type','Material Group desc.':'Material Group'},inplace=True)
#ves_2.drop(columns=['Material Group desc.'],inplace=True)
print(ves_2.shape)
ves_2['First Statistical delivery Date']=ves_2['Last Statistical delivery Date']
ves_2['Plant ID']=ves_2['Plant ID'].astype(int)
ves_2['Supplier No.']=ves_2['Supplier No.'].astype(int)
ves_2['Purchase Order']=ves_2['Purchase Order'].astype(int)
ves_2['Material No.']=ves_2['Material No.'].astype(int)
ves_21=ves_2.merge(ves_cut,on=['Plant ID','Material No.','Supplier No.'],how='inner')
ves_21['Missed_PO']='Yes'
print(ves_21.shape)
ves_21.head(1)

(2092, 25)
(2081, 25)
(830, 29)
Out[55]:

,Plant ID,Plant Name,Supplier No.,VS Name,Purchase Order,PO Item,Material No.,Material Description,MRP controller,Manual Override (Y/N),Purchase Order Scheduled Qty,Delivered Quantity,First GR Date,Last GR Date,Last Statistical delivery Date,On-Time Delivery(%),Quantity Reliability(%),Initial Quality(%),Total Quality(%),Under Delievry Tolerance(%),Over Delivery Tolerance(%),Early Delivery Tolerance(Days),Late Delievry Tolerance(Days),Material_group_type,Material Group,First Statistical delivery Date,Vendor,VM Name,Missed_PO
0,5446,US PL Mt Sterling,100368536,ADM Milling Co,4568052467,10.0,43058751,Flour Wheat Bread Ublch Enriched Bulk,307.0,N,50000.0,43040.0,2021-12-01,2021-12-01,2021-12-01,1.0,0.0,1.0,1.0,10.0,10.0,2.0,1.0,R10,Cereals & Grains,2021-12-01,100037407,Nestrade S.A.,Yes


In [0]:
ves_final=pd.concat([ves_11,ves_21])
ves_final.drop_duplicates(inplace=True)
ves_final['PO Item']=ves_final['PO Item'].astype(int)
print(ves_final.shape)
ves_final['Purchase_order_PO_item'] = ves_final['Purchase Order'].astype(str) + '_' + ves_final['PO Item'].astype(str)
print(ves_final.shape)
ves_final.head(2)

(10823, 29)
(10823, 30)
Out[56]:

,Plant ID,Plant Name,Vendor,VM Name,Supplier No.,VS Name,Purchase Order,Material_group_type,Material Group,Material No.,Material Description,MRP controller,Manual Override (Y/N),First GR Date,Last GR Date,First Statistical delivery Date,Last Statistical delivery Date,Under Delievry Tolerance(%),Over Delivery Tolerance(%),Early Delivery Tolerance(Days),Late Delievry Tolerance(Days),On-Time Delivery(%),Quantity Reliability(%),Initial Quality(%),Total Quality(%),Purchase Order Scheduled Qty,Delivered Quantity,PO Item,Missed_PO,Purchase_order_PO_item
0,5004,US PL SOLON,100037407,Nestrade S.A.,100099158,Leprino Foods Dairy Products Co,4567777068,R01,Dairy Products,22002012,Cheese Mozz PrtSk Pmsn Flv Shd IQF,303,N,2021-11-19,2021-11-19,2021-11-19,2021-11-19,5.0,5.0,2.0,1.0,100.0,100.0,100.0,100.0,19800.0,19800.0,10,No,4567777068_10
1,5004,US PL SOLON,100037407,Nestrade S.A.,100099158,Leprino Foods Dairy Products Co,4568036281,R01,Dairy Products,22002012,Cheese Mozz PrtSk Pmsn Flv Shd IQF,303,N,2021-12-21,2021-12-21,2021-12-21,2021-12-21,5.0,5.0,2.0,1.0,100.0,100.0,100.0,100.0,5400.0,5400.0,10,No,4568036281_10


In [0]:
ves_old=spark.table("model_data_vf_19_01")
ves_old=ves_old.toPandas()
ves_old.head(1)

Out[57]:

,Plant_ID,Plant_Name,Vendor,VM_Name,Purchase_Order,PO_Item,Material_Group,Material_No.,Material_Description,First_GR_Date,Last_GR_Date,First_Statistical_delivery_Date,Delivery_date,Purchase_Order_Scheduled_Qty,Delivered_Quantity,Purchase_order_PO_item,Missed_PO,PO_Create_Date,updated_VS_country,updated_VS_ID,updated_VS_location,updated_VS_region,distance,vendor_percentage,vendor_material_cnt_per_plant,sku_type_single,dom_or_int_int,Material_group_type_R,nestle_managed_freight_yes,mode_of_transport_road,single_or_multisource_single,PO_Create_Month_1,PO_Create_Month_2,PO_Create_Month_3,PO_Create_Month_4,PO_Create_Month_5,PO_Create_Month_6,PO_Create_Month_7,PO_Create_Month_8,PO_Create_Month_9,PO_Create_Month_10,PO_Create_Month_11,PO_Create_Month_12,bus_day_count,lead_time,holiday_weekend_count,OTIF,l3m_date,l12m_date,no.of.complaint_3_months,no.of.complaint_12_months,no.of.complaint_3_months_major,no.of.complaint_3_months_minor,no.of.complaint_12_months_major,no.of.complaint_12_months_minor,no.of.complaint_3_months_open,no.of.complaint_3_months_open_major,no.of.complaint_3_months_open_minor,no.of.Complaint_type_Documentation_l12m,Complaint_type_Documentation_l12m_distinct,no.of.Complaint_type_Packaging_l12m,Complaint_type_Packaging_l12m_distinct,no.of.Complaint_type_Transportation_l12m,Complaint_type_Transportation_l12m_distinct,no.of.Complaint_type_Ingredient_l12m,Complaint_type_Ingredient_l12m_distinct,no.of.Complaint_type_Consumer_l12m,Complaint_type_Consumer_l12m_distinct,no.of.Complaint_type_Foreign_Material_l12m,Complaint_type_Foreign_Material_l12m_distinct,no.of.Complaint_type_Indigenous_Material_l12m,Complaint_type_Indigenous_Material_l12m_distinct,Risk_Level_High,Risk_Level_Low,Risk_Level_Medium,Rating_A,Rating_B,Rating_C,Rating_D,Rating_D_-_FND,Rating_F,Rating_Inactive,Rating_Out_of_Scope,days,Delivered_Quantity_UoM,Type_of_Measure,MovingAverage,POqty_to_AvgDelQty,range_bin_7_to_70,range_bin_70_to_217,range_bin_217_to_326,range_bin_326_to_544,range_bin_544_to_904,range_bin_904_to_1288,range_bin_1288_to_2153,range_bin_2153_to_2874,range_bin_2874_to_15463,Delivery_freq_Daily,Delivery_freq_Half_Yearly,Delivery_freq_Monthly,Delivery_freq_Quarterly,Delivery_freq_Weekly,Delivery_freq_Yearly
0,5258,us pl medford,100364660,international paper company,4562214653,10,corrugated,43572559,blankcorrcircle23-23-23virginliner11.50,2020-01-23,2020-01-23,2020-01-22,2020-01-22,293760.0,293760.0,4562214653_10,No,2020-01-14,us,100855106,ramsey,mn,297.1187,97.51,2,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,6,8,2,1.0,2019-10-16,2019-01-14,15,15,2,13,2,13,0,0,0,0,0,15,1,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,7.183673,IEA,unit,293520.0,1.000818,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0


In [0]:
ves_old_file=ves_old[['updated_VS_ID','updated_VS_location','updated_VS_region','updated_VS_country']].copy()
ves_old_file.drop_duplicates(inplace=True)

In [0]:
order_freq_data='order_freq_19_01'
model_data_final='model_data_vf_19_01'
historical_invt_raw_data ='Historical_inventory_rawData_169_03_01'
inventory_consumption_mb51='ConsumptionMB51_ROH_ZPCK_28_12'

In [0]:
#Dropping records where First Statistical delivery Date is null
ves_final.dropna(subset=['First Statistical delivery Date'], inplace=True)
# VES_old_data = VES_old_data[['Plant', 'Purchase_order_PO_item', 'Material No.', 'PO Create Date']].drop_duplicates()
# VES_old_data = VES_old_data.drop_duplicates(subset=['Plant', 'Purchase_order_PO_item', 'Material No.'], keep='last')
# VES_old_data.rename(columns={'Plant':'Plant ID'}, inplace=True)

#Cleaning PO Create Date data to merge with VES data for populating PO_Create_Date in VES data
# PO_create_date.drop(PO_create_date.columns[0],axis=1,inplace=True)
PO_create_date.rename(columns={'Plant - Key':'Plant ID', 'Material - Key':'Material No.', 
                               'PO Item Number':'PO Item', 'PO Number':'Purchase Order'}, inplace=True)

#Creating unique key of PO number and PO Item to get PO_Create_Date from VES, any missing values will be taken from Spend Analysis
PO_create_date['Purchase_order_PO_item'] = PO_create_date['Purchase Order'].astype(str) + '_' + PO_create_date['PO Item'].astype(str)
PO_create_date = PO_create_date.drop_duplicates(subset=['Purchase_order_PO_item', 'Plant ID', 'Material No.'], keep='last')
PO_create_date.rename(columns={'PO Item Creation Date':'PO Create Date'}, inplace=True)

#Converting date columns to datetime object
PO_create_date['PO Create Date'] = pd.to_datetime(PO_create_date['PO Create Date'])
PO_create_date.drop(columns=['Purchase Order','PO Item'],inplace=True)
# VES_old_data['PO Create Date'] = pd.to_datetime(VES_old_data['PO Create Date'])

#Creating a single PO Create Date file
# PO_create_date_final = pd.concat([PO_create_date[['Purchase_order_PO_item', 'Plant ID', 'Material No.', 'PO Create Date']], 
#                                   VES_old_data[['Purchase_order_PO_item', 'Plant ID', 'Material No.', 'PO Create Date']]], ignore_index=True)
# PO_create_date_final = PO_create_date_final.drop_duplicates()

#Merging PO Create Date file with VES data to Populate VES data with PO Create Date
ves_final_w_PO_create_date = ves_final.merge(PO_create_date, on=['Purchase_order_PO_item', 'Plant ID', 'Material No.'], how='left')

In [0]:
print(ves_final_w_PO_create_date.shape)
ves_final_w_PO_create_date.head(1)

(10823, 31)
Out[61]:

,Plant ID,Plant Name,Vendor,VM Name,Supplier No.,VS Name,Purchase Order,Material_group_type,Material Group,Material No.,Material Description,MRP controller,Manual Override (Y/N),First GR Date,Last GR Date,First Statistical delivery Date,Last Statistical delivery Date,Under Delievry Tolerance(%),Over Delivery Tolerance(%),Early Delivery Tolerance(Days),Late Delievry Tolerance(Days),On-Time Delivery(%),Quantity Reliability(%),Initial Quality(%),Total Quality(%),Purchase Order Scheduled Qty,Delivered Quantity,PO Item,Missed_PO,Purchase_order_PO_item,PO Create Date
0,5004,US PL SOLON,100037407,Nestrade S.A.,100099158,Leprino Foods Dairy Products Co,4567777068,R01,Dairy Products,22002012,Cheese Mozz PrtSk Pmsn Flv Shd IQF,303,N,2021-11-19,2021-11-19,2021-11-19,2021-11-19,5.0,5.0,2.0,1.0,100.0,100.0,100.0,100.0,19800.0,19800.0,10,No,4567777068_10,NaT


In [0]:
#2: Changing columns names to write as a spark table
col_names = ['_'.join(col.split(' ')) for col in ves_final_w_PO_create_date.columns.tolist()]
ves_final_w_PO_create_date.columns = col_names
#Changing datatypes of some columns to string to make them uniform
ves_final_w_PO_create_date['MRP_controller'] = ves_final_w_PO_create_date['MRP_controller'].astype(str)
ves_final_w_PO_create_date['On-Time_Delivery(%)'] = ves_final_w_PO_create_date['On-Time_Delivery(%)'].astype(str)
ves_final_w_PO_create_date['Quantity_Reliability(%)'] = ves_final_w_PO_create_date['Quantity_Reliability(%)'].astype(str)
ves_final_w_PO_create_date['Initial_Quality(%)'] = ves_final_w_PO_create_date['Initial_Quality(%)'].astype(str)
ves_final_w_PO_create_date['Total_Quality(%)'] = ves_final_w_PO_create_date['Total_Quality(%)'].astype(str)
ves_final_w_PO_create_date['VM_Name'] = ves_final_w_PO_create_date['VM_Name'].str.lower()
ves_final_w_PO_create_date['Plant_Name'] = ves_final_w_PO_create_date['Plant_Name'].str.lower()
ves_final_w_PO_create_date['VS_Name'] = ves_final_w_PO_create_date['VS_Name'].str.lower()
ves_final_w_PO_create_date['Material_Group'] = ves_final_w_PO_create_date['Material_Group'].str.lower()
ves_final_w_PO_create_date['Material_Description'] = ves_final_w_PO_create_date['Material_Description'].str.lower()
ves_final_w_PO_create_date.rename(columns={'Manual_Override_(Y/N)':'Manual_Override', 'Under_Delievry_Tolerance(%)':'Under_Delievry_Tolerance_pct', 
                                           'Over_Delivery_Tolerance(%)':'Over_Delivery_Tolerance_pct', 
                                           'Early_Delivery_Tolerance(Days)':'Early_Delivery_Tolerance_Days', 
                                           'Late_Delievry_Tolerance(Days)':'Late_Delievry_Tolerance_Days', 
                                           'On-Time_Delivery(%)':'On-Time_Delivery_pct', 'Quantity_Reliability(%)':'Quantity_Reliability_pct', 
                                           'Initial_Quality(%)':'Initial_Quality_pct', 'Total_Quality(%)':'Total_Quality_pct'}, inplace=True)

In [0]:
#3: Cleaning Date columns
#Renaming Last_Statistical_delivery_Date to Delivery_date
ves_final_w_PO_create_date.rename(columns={'Last_Statistical_delivery_Date':'Delivery_date'}, inplace=True)

#Replacing '#' with nans in Date columns to convert to datetime
ves_final_w_PO_create_date['Delivery_date'].replace('#', inplace=True)#check if all these can be added in 1 line
ves_final_w_PO_create_date['First_GR_Date'].replace('#', inplace=True)
ves_final_w_PO_create_date['Last_GR_Date'].replace('#', inplace=True)
ves_final_w_PO_create_date['First_Statistical_delivery_Date'].replace('#', inplace=True)
ves_final_w_PO_create_date['Delivery_date'] = pd.to_datetime(ves_final_w_PO_create_date['Delivery_date'])
ves_final_w_PO_create_date['First_GR_Date'] = pd.to_datetime(ves_final_w_PO_create_date['First_GR_Date'])
ves_final_w_PO_create_date['Last_GR_Date'] = pd.to_datetime(ves_final_w_PO_create_date['Last_GR_Date'])
ves_final_w_PO_create_date['First_Statistical_delivery_Date'] = pd.to_datetime(ves_final_w_PO_create_date['First_Statistical_delivery_Date'])
ves_final_w_PO_create_date['PO_Create_Date'] = pd.to_datetime(ves_final_w_PO_create_date['PO_Create_Date'])

#Replacing nans with a dummy date for PO Create Date: '1888-01-01'
ves_final_w_PO_create_date['PO_Create_Date'].fillna(pd.to_datetime('1888-01-01'), inplace = True)

#Filtering VES for in scope materials
ves = ves_final_w_PO_create_date[ves_final_w_PO_create_date['Material_No.'].isin(in_scope_material['Material'].unique().tolist())]

In [0]:
print(ves.shape)
ves.head(1)

(10823, 31)
Out[64]:

,Plant_ID,Plant_Name,Vendor,VM_Name,Supplier_No.,VS_Name,Purchase_Order,Material_group_type,Material_Group,Material_No.,Material_Description,MRP_controller,Manual_Override,First_GR_Date,Last_GR_Date,First_Statistical_delivery_Date,Delivery_date,Under_Delievry_Tolerance_pct,Over_Delivery_Tolerance_pct,Early_Delivery_Tolerance_Days,Late_Delievry_Tolerance_Days,On-Time_Delivery_pct,Quantity_Reliability_pct,Initial_Quality_pct,Total_Quality_pct,Purchase_Order_Scheduled_Qty,Delivered_Quantity,PO_Item,Missed_PO,Purchase_order_PO_item,PO_Create_Date
0,5004,us pl solon,100037407,nestrade s.a.,100099158,leprino foods dairy products co,4567777068,R01,dairy products,22002012,cheese mozz prtsk pmsn flv shd iqf,303,N,2021-11-19,2021-11-19,2021-11-19,2021-11-19,5.0,5.0,2.0,1.0,100.0,100.0,100.0,100.0,19800.0,19800.0,10,No,4567777068_10,1888-01-01


In [0]:
# #1: Vendor Subcontractor location, PO Create Date from Spend Analysis
# #Loop for cleaning the Spend Analysis files
# loc_df = []
# for file in [spend_analy]:
#   print(file)
#   spend_analysis_xls = pd.ExcelFile(file)
  
#   if file == spend_analy:
#     spend_analysis_sheet_names = ['5004 SOLON', '5878 JONESBORO', '5955 GAFFNEY', '5243 MOSS LANDING', '5959 SPRINGVILLE']
#   elif file == spend_analy1:
#     spend_analysis_sheet_names = ['5259 LIttle Chute Pack 2020', '5258 MEDFORD_Pack', '5258 MEDFORD_Raw', 
#                                      '5446 MT STERLING Raw', '5446 MT STERLING Pack']
#   else:
#     spend_analysis_sheet_names = ['Sheet 2']
    
#   for sheet in spend_analysis_sheet_names:
#     print(sheet)
#     spend_analysis = pd.read_excel(spend_analysis_xls, sheet, skiprows=3, index_col=0)
    
#     #Renaming columns to match with VES data
#     spend_analysis.rename(columns={'Plant - Key': 'Plant ID','Material - Key':'Material_No.',
#                                    'PO Number - Key':'Purchase_Order','GR Vendor-Site - Key':'sub_id',
#                                    'PO Item Creation Date':'PO_create_date','PO Item Number':'PO_Item',
#                                    'GR Vendor-Site - CAM: City District (Key)':'sub_loc',
#                                    'GR Vendor-Site - Region (Key)':'sub_reg',
#                                    'Supplying Vendor - CAM: City District (Key)':'VS_location', 
#                                    'Supplying Vendor - Region (Key)':'VS_region', 
#                                    'Supplying Vendor - Key':'Supplier No.',
#                                    'Batch Vendor-Site - Key':'batch_vs_id',
#                                    'Batch Vendor-Site - CAM: City District (Key)':'batch_loc',
#                                    'Batch Vendor-Site - Region (Key)':'batch_reg'},inplace=True)
    
#     #Forward filling some columns since they are merged in the raw file
#     spend_analysis['PO_create_date'] = spend_analysis['PO_create_date'].fillna(method='ffill')
#     spend_analysis['PO_Item'] = spend_analysis['PO_Item'].fillna(method='ffill')
    
#     #Subsetting spend analysis data for required columns
#     vs_loc = spend_analysis[['Plant ID','Purchase_Order','PO_Item','Material_No.','PO_create_date','sub_id','sub_loc','sub_reg','Supplier No.', 'VS_location', 'VS_region', 'Supplying Vendor - Country (Key)','batch_vs_id','batch_loc','batch_reg']].drop_duplicates()
    
#     #Filtering each sheet data for in scope materials
#     vs_loc = vs_loc[vs_loc['Material_No.'].isin(in_scope_material['Material'].unique().tolist())]
#     loc_df.append(vs_loc)

# vs_loc_final = pd.concat(loc_df, ignore_index=True)

In [0]:
#1: Vendor Subcontractor location, PO Create Date from Spend Analysis
#Loop for cleaning the Spend Analysis files
loc_df = []
# for file in [spend_analy]:
#   print(file)
#   spend_analysis_xls = pd.ExcelFile(file)
  
#   if file == spend_analy:
#     spend_analysis_sheet_names = ['5004 SOLON', '5878 JONESBORO', '5955 GAFFNEY', '5243 MOSS LANDING', '5959 SPRINGVILLE']
#   elif file == spend_analy1:
#     spend_analysis_sheet_names = ['5259 LIttle Chute Pack 2020', '5258 MEDFORD_Pack', '5258 MEDFORD_Raw', 
#                                      '5446 MT STERLING Raw', '5446 MT STERLING Pack']
#   else:
#     spend_analysis_sheet_names = ['Sheet 2']
    
#   for sheet in spend_analysis_sheet_names:
#     print(sheet)
spend_analysis = pd.read_csv('enter file path to Spend_Analysis_16_03_cleaned.xlsx', index_col=0)

#Renaming columns to match with VES data
spend_analysis.rename(columns={'Plant - Key': 'Plant ID','Material - Key':'Material_No.',
                               'PO Number - Key':'Purchase_Order','GR Vendor-Site - Key':'sub_id',
                               'PO Item Creation Date':'PO_create_date','PO Item Number':'PO_Item',
                               'GR Vendor-Site - CAM: City District (Key)':'sub_loc',
                               'GR Vendor-Site - Region (Key)':'sub_reg',
                               'Supplying Vendor - CAM: City District (Key)':'VS_location', 
                               'Supplying Vendor - Region (Key)':'VS_region', 
                               'Supplying Vendor - Key':'Supplier No.',
                               'Batch Vendor-Site - Key':'batch_vs_id',
                               'Batch Vendor-Site - CAM: City District (Key)':'batch_loc',
                               'Batch Vendor-Site - Region (Key)':'batch_reg'},inplace=True)

#Forward filling some columns since they are merged in the raw file
spend_analysis['PO_create_date'] = spend_analysis['PO_create_date'].fillna(method='ffill')
spend_analysis['PO_Item'] = spend_analysis['PO_Item'].fillna(method='ffill')

#Subsetting spend analysis data for required columns
vs_loc = spend_analysis[['Plant ID','Purchase_Order','PO_Item','Material_No.','PO_create_date','sub_id','sub_loc','sub_reg','Supplier No.', 'VS_location', 'VS_region', 'Supplying Vendor - Country (Key)','batch_vs_id','batch_loc','batch_reg']].drop_duplicates()

#Filtering each sheet data for in scope materials
vs_loc = vs_loc[vs_loc['Material_No.'].isin(in_scope_material['Material'].unique().tolist())]
loc_df.append(vs_loc)

vs_loc_final = pd.concat(loc_df, ignore_index=True)

In [0]:
print(vs_loc_final.shape)
vs_loc_final.head(1)

(13966, 15)
Out[67]:

,Plant ID,Purchase_Order,PO_Item,Material_No.,PO_create_date,sub_id,sub_loc,sub_reg,Supplier No.,VS_location,VS_region,Supplying Vendor - Country (Key),batch_vs_id,batch_loc,batch_reg
0,5004.0,4.567373e+09,10.0,22013995.0,2021-09-02,100364660,shelby,tn,100364660,shelby,tn,us,100364660,shelby,tn


In [0]:
spend_analysis[sorted(spend_analysis.columns.tolist())].head(1)

Out[68]:

,Batch Vendor-Site - Country (Key),Batch Vendor-Site - Postal Code (Key),Contract Reference Item Number,Contract Reference Number - Key,GR Mat Doc,GR Vendor-Site,GR Vendor-Site - Country (Key),GR Vendor-Site - Postal Code (Key),Material - Text,Material Document,Material_No.,PO_Item,PO_create_date,Plant - Text,Plant ID,Posting date,Purchase_Order,Spend Group,Supplier No.,Supplying Vendor - Country (Key),Supplying Vendor - Location (Key),Supplying Vendor - Postal Code (Key),Supplying Vendor - Text,VS_location,VS_region,Vendor - Key,Vendor - Text,batch_loc,batch_reg,batch_vs_id,sub_id,sub_loc,sub_reg
0,us,02852-7520,1,A9APRR3103/MA076231,8672392559,Not assigned,us,#,Case Corr STFR Gnrc 12xBoil In Bag Pouch,5.335890e+09,22013995.0,10.0,2021-09-02,US PL SOLON,5004.0,2020-03-18,4.567373e+09,Packaging,100364660,us,shelby,02852-7520,International Paper Company,shelby,tn,100364660.0,International Paper Company,shelby,tn,100364660,100364660,shelby,tn


In [0]:
#Creating fields for Vendor Site (VS) and Vendor locations
vs_loc_final['updated_VS_ID']=vs_loc_final['sub_id']
vs_loc_final['updated_VS_location']=vs_loc_final['sub_loc']
vs_loc_final['updated_VS_region']=vs_loc_final['sub_reg']

#There are 3 available field groups for getting the VS ID and location, Following steps are to iterate through these 3 fields to identify non missing values across the 3 fields, namely: GR, Batch and Supplier in the given order of priority (GR>Batch>Supplier)

#First step: logic for obtaining GR
vs_id_final=vs_loc_final.copy()

#Removing records which contain invalid characters in VS columns
vs_id_final.drop(vs_id_final.loc[(vs_id_final['sub_id']=='#')|(vs_id_final['sub_id']=='*')].index,axis=0,inplace=True)
vs_id_final.drop(vs_id_final[vs_id_final['updated_VS_ID']=='#100921243'].index,axis=0,inplace=True)
vs_id_final['updated_VS_ID']=vs_id_final['updated_VS_ID'].astype(np.int64)
vs_id_final.drop(columns=['sub_id','sub_loc','sub_reg','Supplier No.', 'VS_location', 'VS_region', 'batch_vs_id', 'batch_loc', 'batch_reg'], 
                 axis=1,inplace=True)
vs_id_final.drop_duplicates(inplace=True)
vs_id_final.rename(columns={'Plant ID':'Plant_ID','PO_create_date':'PO_Create_Date'},inplace=True)
vs_id_final['PO_Create_Date']=pd.to_datetime(vs_id_final['PO_Create_Date'])
vs_id_final['PO_Item']=vs_id_final['PO_Item'].astype(np.int64)

In [0]:
vs_id_final['Plant_ID']=vs_id_final['Plant_ID'].astype(int)
vs_id_final['Purchase_Order']=vs_id_final['Purchase_Order'].astype(int)
vs_id_final['Material_No.']=vs_id_final['Material_No.'].astype(int)
print(vs_id_final.shape)
vs_id_final.head(1)

(13966, 9)
Out[70]:

,Plant_ID,Purchase_Order,PO_Item,Material_No.,PO_Create_Date,Supplying Vendor - Country (Key),updated_VS_ID,updated_VS_location,updated_VS_region
0,5004,4567372919,10,22013995,2021-09-02,us,100364660,shelby,tn


In [0]:
vs_id_final.drop_duplicates(subset=['Plant_ID','Purchase_Order','PO_Item','Material_No.','PO_Create_Date'],inplace=True)
print(vs_id_final.shape)
vs_id_final.head(1)

(10618, 9)
Out[71]:

,Plant_ID,Purchase_Order,PO_Item,Material_No.,PO_Create_Date,Supplying Vendor - Country (Key),updated_VS_ID,updated_VS_location,updated_VS_region
0,5004,4567372919,10,22013995,2021-09-02,us,100364660,shelby,tn


In [0]:
#Obtain correct PO date from Spend Analysis for records with 1888-01-01
podate=[]
for pod in ves['Purchase_Order'][ves['PO_Create_Date']=='1888-01-01']:
  a=vs_loc_final[vs_loc_final['Purchase_Order']==pod].reset_index()
  if a['PO_create_date'].shape[0]>=1:
    podate.append(a['PO_create_date'][0])
  else:
    podate.append('1888-01-01')
ves['PO_Create_Date'][ves['PO_Create_Date']=='1888-01-01']=podate
ves['PO_Create_Date']=pd.to_datetime(ves['PO_Create_Date'])
print(ves.shape)
#Merging GR iteration with VES data
ves_f=pd.merge(ves,vs_id_final,on=('Plant_ID','Purchase_Order','PO_Item','Material_No.','PO_Create_Date'),how='left')

(10823, 31)

In [0]:
print(ves_f.shape)
ves_f.head(1)

(10823, 35)
Out[73]:

,Plant_ID,Plant_Name,Vendor,VM_Name,Supplier_No.,VS_Name,Purchase_Order,Material_group_type,Material_Group,Material_No.,Material_Description,MRP_controller,Manual_Override,First_GR_Date,Last_GR_Date,First_Statistical_delivery_Date,Delivery_date,Under_Delievry_Tolerance_pct,Over_Delivery_Tolerance_pct,Early_Delivery_Tolerance_Days,Late_Delievry_Tolerance_Days,On-Time_Delivery_pct,Quantity_Reliability_pct,Initial_Quality_pct,Total_Quality_pct,Purchase_Order_Scheduled_Qty,Delivered_Quantity,PO_Item,Missed_PO,Purchase_order_PO_item,PO_Create_Date,Supplying Vendor - Country (Key),updated_VS_ID,updated_VS_location,updated_VS_region
0,5004,us pl solon,100037407,nestrade s.a.,100099158,leprino foods dairy products co,4567777068,R01,dairy products,22002012,cheese mozz prtsk pmsn flv shd iqf,303,N,2021-11-19,2021-11-19,2021-11-19,2021-11-19,5.0,5.0,2.0,1.0,100.0,100.0,100.0,100.0,19800.0,19800.0,10,No,4567777068_10,2022-01-04,us,100907802.0,san joaquin,ca


In [0]:
ves_f.isnull().sum()

Out[74]: Plant_ID 0
Plant_Name 0
Vendor 0
VM_Name 0
Supplier_No. 0
VS_Name 0
Purchase_Order 0
Material_group_type 0
Material_Group 0
Material_No. 0
Material_Description 0
MRP_controller 0
Manual_Override 0
First_GR_Date 0
Last_GR_Date 0
First_Statistical_delivery_Date 0
Delivery_date 0
Under_Delievry_Tolerance_pct 0
Over_Delivery_Tolerance_pct 0
Early_Delivery_Tolerance_Days 0
Late_Delievry_Tolerance_Days 0
On-Time_Delivery_pct 0
Quantity_Reliability_pct 0
Initial_Quality_pct 0
Total_Quality_pct 0
Purchase_Order_Scheduled_Qty 0
Delivered_Quantity 0
PO_Item 0
Missed_PO 0
Purchase_order_PO_item 0
PO_Create_Date 0
Supplying Vendor - Country (Key) 1350
updated_VS_ID 1350
updated_VS_location 1350
updated_VS_region 1397
dtype: int64

In [0]:
#Splitting w.r.t Supplier & Batch for VS_id, VS_location, VS_region
vs_loc_final_v1=vs_loc_final[(vs_loc_final['updated_VS_ID']=='#') | (vs_loc_final['updated_VS_ID']=='*')]
vs_loc_final_sup=vs_loc_final_v1[['Plant ID','Purchase_Order','PO_Item','Material_No.','PO_create_date','Supplier No.','VS_location','VS_region','Supplying Vendor - Country (Key)']]
vs_loc_final_bat=vs_loc_final_v1[['Plant ID', 'Purchase_Order', 'PO_Item', 'Material_No.', 'PO_create_date', 'batch_vs_id', 'batch_loc', 
                                  'batch_reg', 'Supplying Vendor - Country (Key)']]
vs_loc_final_sup.drop_duplicates(inplace=True)
vs_loc_final_bat.drop_duplicates(inplace=True)
vs_loc_final_bat.drop(vs_loc_final_bat[vs_loc_final_bat['batch_vs_id']=='#'].index,axis=0,inplace=True)
vs_loc_final_bat.drop(vs_loc_final_bat[(vs_loc_final_bat['batch_vs_id']=='MISSING') | 
                                       (vs_loc_final_bat['batch_vs_id']==' *')].index,axis=0,inplace=True)
vs_loc_final_bat['batch_vs_id']=vs_loc_final_bat['batch_vs_id'].astype(int)
vs_loc_final_sup['PO_Item']=vs_loc_final_sup['PO_Item'].astype(np.int64)
vs_loc_final_bat['PO_Item']=vs_loc_final_bat['PO_Item'].astype(np.int64)
vs_loc_final_sup['PO_create_date']=pd.to_datetime(vs_loc_final_sup['PO_create_date'])
vs_loc_final_bat['PO_create_date']=pd.to_datetime(vs_loc_final_bat['PO_create_date'])
vs_loc_final_sup_v1=vs_loc_final_sup[['Supplier No.','VS_location','VS_region','Supplying Vendor - Country (Key)']].copy()
vs_loc_final_sup_v1=vs_loc_final_sup_v1.drop_duplicates()
vs_loc_final_sup_v1.rename(columns={'Supplier No.':'updated_VS_ID'},inplace=True)

In [0]:
print(vs_loc_final_bat.shape)
vs_loc_final_bat.head(1)

(0, 9)
Out[76]:

,Plant ID,Purchase_Order,PO_Item,Material_No.,PO_create_date,batch_vs_id,batch_loc,batch_reg,Supplying Vendor - Country (Key)


In [0]:
#Second and third step: Logic for obtaining Batch and then Supplier
#Creating VS info from Batch VS info, if Batch not available then Supplier info is taken, if Supplier not available then filling as null
ves_f_v1=ves_f[ves_f['updated_VS_ID'].isnull()]
vs_id=[]
loc=[]
reg=[]
coun=[]
for p_id,po,mn,pi,pds,sn in zip(ves_f_v1['Plant_ID'],ves_f_v1['Purchase_Order'],ves_f_v1['Material_No.'],ves_f_v1['PO_Item'],ves_f_v1['PO_Create_Date'],ves_f_v1['Supplier_No.']):
  a=vs_loc_final_bat[(vs_loc_final_bat['Plant ID']==p_id) & (vs_loc_final_bat['Purchase_Order']==po) & (vs_loc_final_bat['Material_No.']==mn) & (vs_loc_final_bat['PO_Item']==pi) & (vs_loc_final_bat['PO_create_date']==pds)].reset_index()
  b=vs_loc_final_sup[(vs_loc_final_sup['Plant ID']==p_id) & (vs_loc_final_sup['Purchase_Order']==po) & (vs_loc_final_sup['Material_No.']==mn) & (vs_loc_final_sup['PO_Item']==pi) & (vs_loc_final_sup['PO_create_date']==pds)].reset_index()
  if a['batch_vs_id'].shape[0]>=1:
    vs_id.append(a['batch_vs_id'][0])
    loc.append(a['batch_loc'][0])
    reg.append(a['batch_reg'][0])
    coun.append(a['Supplying Vendor - Country (Key)'][0])
  elif b['Supplier No.'].shape[0]>=1:
    vs_id.append(b['Supplier No.'][0])
    loc.append(b['VS_location'][0])
    reg.append(b['VS_region'][0])
    coun.append(b['Supplying Vendor - Country (Key)'][0])
  else:
    vs_id.append('null')
    loc.append('null')
    reg.append('null')
    coun.append('null')

#Updating VS info created above in VES 
ves_f_v1['updated_VS_ID']=vs_id
ves_f_v1['updated_VS_location']=loc
ves_f_v1['updated_VS_region']=reg
ves_f_v1['Supplying Vendor - Country (Key)']=coun

In [0]:
#updating VS ID with null substituted with Supplier no from ves
ves_f_v2=ves_f_v1[ves_f_v1['updated_VS_ID']=='null']
ves_f_v2.loc[(ves_f_v2['updated_VS_ID']=='null'),'updated_VS_ID']=ves_f_v2['Supplier_No.']

#If after all the iterations, VS location is not available, then taking VS location from Spend Analysis Supplier locations
ves_f_sup=pd.merge(ves_f_v2,vs_loc_final_sup_v1,how='left',on=['updated_VS_ID'])
ves_f_sup.drop(columns=['updated_VS_location','updated_VS_region','Supplying Vendor - Country (Key)_x'],axis=0,inplace=True)
ves_f_sup.rename(columns={'VS_location':'updated_VS_location','VS_region':'updated_VS_region','Supplying Vendor - Country (Key)_y':'Supplying Vendor - Country (Key)'},inplace=True)

#Creating cuts for batch, supplier and GR from Spend Analysis
ves_f_batch_sup=ves_f_v1[ves_f_v1['updated_VS_ID']!='null']
ves_f_gr=ves_f[ves_f['updated_VS_ID'].notnull()]

#Creating final VES with updated VS Info
ves_finale=pd.concat([ves_f_gr,ves_f_batch_sup,ves_f_sup],axis=0)
cols=['_'.join(i.split(' ')) for i in ves_finale.columns.tolist()]
ves_finale.columns = cols
ves_finale.rename(columns={'Supplying_Vendor_-_Country_(Key)':'updated_VS_country'},inplace=True)

#Dropping records with VS ID equal to 0 or 7
ves_finale.drop(ves_finale[(ves_finale['updated_VS_ID']==0)|(ves_finale['updated_VS_ID']==7)].index,axis=0,inplace=True)

In [0]:
print(ves_finale.shape)
ves_finale.head(1)

(10823, 35)
Out[79]:

,Plant_ID,Plant_Name,Vendor,VM_Name,Supplier_No.,VS_Name,Purchase_Order,Material_group_type,Material_Group,Material_No.,Material_Description,MRP_controller,Manual_Override,First_GR_Date,Last_GR_Date,First_Statistical_delivery_Date,Delivery_date,Under_Delievry_Tolerance_pct,Over_Delivery_Tolerance_pct,Early_Delivery_Tolerance_Days,Late_Delievry_Tolerance_Days,On-Time_Delivery_pct,Quantity_Reliability_pct,Initial_Quality_pct,Total_Quality_pct,Purchase_Order_Scheduled_Qty,Delivered_Quantity,PO_Item,Missed_PO,Purchase_order_PO_item,PO_Create_Date,updated_VS_country,updated_VS_ID,updated_VS_location,updated_VS_region
0,5004,us pl solon,100037407,nestrade s.a.,100099158,leprino foods dairy products co,4567777068,R01,dairy products,22002012,cheese mozz prtsk pmsn flv shd iqf,303,N,2021-11-19,2021-11-19,2021-11-19,2021-11-19,5.0,5.0,2.0,1.0,100.0,100.0,100.0,100.0,19800.0,19800.0,10,No,4567777068_10,2022-01-04,us,100907802.0,san joaquin,ca


In [0]:
ves_finale_1=ves_finale[ves_finale['updated_VS_country'].isnull()==False]
ves_finale_2=ves_finale[ves_finale['updated_VS_country'].isnull()==True]
ves_finale_2['updated_VS_ID']=ves_finale_2['updated_VS_ID'].astype(int)

In [0]:
for vs_id,vs_loc,vs_reg,vs_cnty,indices in zip(ves_finale_2['updated_VS_ID'],ves_finale_2['updated_VS_location'],ves_finale_2['updated_VS_region'],ves_finale_2['updated_VS_country'],ves.index):
  try:
    loc=ves_old_file[ves_old_file['updated_VS_ID']==vs_id]['updated_VS_location'].values[0]
    reg=ves_old_file[ves_old_file['updated_VS_ID']==vs_id]['updated_VS_region'].values[0]
    conty=ves_old_file[ves_old_file['updated_VS_ID']==vs_id]['updated_VS_country'].values[0]
    ves_finale_2['updated_VS_location'][indices]=loc
    ves_finale_2['updated_VS_region'][indices]=reg
    ves_finale_2['updated_VS_country'][indices]=conty
  except:
    ves_finale_2['updated_VS_location'][indices]=np.nan
    ves_finale_2['updated_VS_region'][indices]=np.nan
    ves_finale_2['updated_VS_country'][indices]=np.nan

In [0]:
ves_finale_2.isnull().sum()

Out[82]: Plant_ID 0
Plant_Name 0
Vendor 0
VM_Name 0
Supplier_No. 0
VS_Name 0
Purchase_Order 0
Material_group_type 0
Material_Group 0
Material_No. 0
Material_Description 0
MRP_controller 0
Manual_Override 0
First_GR_Date 0
Last_GR_Date 0
First_Statistical_delivery_Date 0
Delivery_date 0
Under_Delievry_Tolerance_pct 0
Over_Delivery_Tolerance_pct 0
Early_Delivery_Tolerance_Days 0
Late_Delievry_Tolerance_Days 0
On-Time_Delivery_pct 0
Quantity_Reliability_pct 0
Initial_Quality_pct 0
Total_Quality_pct 0
Purchase_Order_Scheduled_Qty 0
Delivered_Quantity 0
PO_Item 0
Missed_PO 0
Purchase_order_PO_item 0
PO_Create_Date 0
updated_VS_country 226
updated_VS_ID 0
updated_VS_location 226
updated_VS_region 226
dtype: int64

In [0]:
ves_finale_2.loc[(ves_finale_2['Delivery_date']>='2022-03-01') & (ves_finale_2['updated_VS_country'].isnull()==True)][['Plant_ID','Material_No.']].drop_duplicates()

Out[86]:

,Plant_ID,Material_No.


In [0]:
ves_finale_2.loc[(ves_finale_2['Delivery_date']>='2022-03-01') & (ves_finale_2['updated_VS_country'].isnull()==True) & (ves_finale_2['Plant_ID']==5959) & (ves_finale_2['Material_No.']==22002573),'updated_VS_country']='es'
ves_finale_2.loc[(ves_finale_2['Delivery_date']>='2022-03-01') & (ves_finale_2['updated_VS_country'].isnull()==True) & (ves_finale_2['Plant_ID']==5959) & (ves_finale_2['Material_No.']==22002573),'updated_VS_ID']=100184989
ves_finale_2.loc[(ves_finale_2['Delivery_date']>='2022-03-01') & (ves_finale_2['updated_VS_country'].isnull()==True) & (ves_finale_2['Plant_ID']==5959) & (ves_finale_2['Material_No.']==22002573),'updated_VS_location']='cabezo de torres murcia'
ves_finale_2.loc[(ves_finale_2['Delivery_date']>='2022-03-01') & (ves_finale_2['updated_VS_country'].isnull()==True) & (ves_finale_2['Plant_ID']==5959) & (ves_finale_2['Material_No.']==22002573),'updated_VS_region']='30'
ves_finale_2.loc[(ves_finale_2['Delivery_date']>='2022-03-01') & (ves_finale_2['updated_VS_country'].isnull()==True) & (ves_finale_2['Plant_ID']==5959) & (ves_finale_2['Material_No.']==22001316),'updated_VS_country']='us'
ves_finale_2.loc[(ves_finale_2['Delivery_date']>='2022-03-01') & (ves_finale_2['updated_VS_country'].isnull()==True) & (ves_finale_2['Plant_ID']==5959) & (ves_finale_2['Material_No.']==22001316),'updated_VS_ID']=100937695
ves_finale_2.loc[(ves_finale_2['Delivery_date']>='2022-03-01') & (ves_finale_2['updated_VS_country'].isnull()==True) & (ves_finale_2['Plant_ID']==5959) & (ves_finale_2['Material_No.']==22001316),'updated_VS_location']='madison'
ves_finale_2.loc[(ves_finale_2['Delivery_date']>='2022-03-01') & (ves_finale_2['updated_VS_country'].isnull()==True) & (ves_finale_2['Plant_ID']==5959) & (ves_finale_2['Material_No.']==22001316),'updated_VS_region']='id'

In [0]:
ves_finale_2['updated_VS_country'][97]='us'
ves_finale_2['updated_VS_ID'][97]=100922708
ves_finale_2['updated_VS_location'][97]='newport news city'
ves_finale_2['updated_VS_region'][97]='va'
ves_finale_2.dropna(inplace=True)

In [0]:
ves_finale=pd.concat([ves_finale_1,ves_finale_2])
print(ves_finale.shape)
ves_finale.head()

(10598, 35)
Out[87]:

,Plant_ID,Plant_Name,Vendor,VM_Name,Supplier_No.,VS_Name,Purchase_Order,Material_group_type,Material_Group,Material_No.,Material_Description,MRP_controller,Manual_Override,First_GR_Date,Last_GR_Date,First_Statistical_delivery_Date,Delivery_date,Under_Delievry_Tolerance_pct,Over_Delivery_Tolerance_pct,Early_Delivery_Tolerance_Days,Late_Delievry_Tolerance_Days,On-Time_Delivery_pct,Quantity_Reliability_pct,Initial_Quality_pct,Total_Quality_pct,Purchase_Order_Scheduled_Qty,Delivered_Quantity,PO_Item,Missed_PO,Purchase_order_PO_item,PO_Create_Date,updated_VS_country,updated_VS_ID,updated_VS_location,updated_VS_region
0,5004,us pl solon,100037407,nestrade s.a.,100099158,leprino foods dairy products co,4567777068,R01,dairy products,22002012,cheese mozz prtsk pmsn flv shd iqf,303,N,2021-11-19,2021-11-19,2021-11-19,2021-11-19,5.0,5.0,2.0,1.0,100.0,100.0,100.0,100.0,19800.00,19800.00,10,No,4567777068_10,2022-01-04,us,100907802.0,san joaquin,ca
1,5004,us pl solon,100037407,nestrade s.a.,100099158,leprino foods dairy products co,4568036281,R01,dairy products,22002012,cheese mozz prtsk pmsn flv shd iqf,303,N,2021-12-21,2021-12-21,2021-12-21,2021-12-21,5.0,5.0,2.0,1.0,100.0,100.0,100.0,100.0,5400.00,5400.00,10,No,4568036281_10,2022-01-04,us,100907802.0,san joaquin,ca
2,5004,us pl solon,100037407,nestrade s.a.,100099158,leprino foods dairy products co,4568305516,R01,dairy products,22002012,cheese mozz prtsk pmsn flv shd iqf,303,N,2022-01-29,2022-01-29,2022-02-08,2022-02-08,5.0,5.0,2.0,1.0,0.0,100.0,100.0,100.0,18000.00,18000.00,10,No,4568305516_10,2022-01-04,us,100907802.0,san joaquin,ca
3,5004,us pl solon,100037407,nestrade s.a.,100099158,leprino foods dairy products co,4568323452,R01,dairy products,22002012,cheese mozz prtsk pmsn flv shd iqf,303,N,2022-01-24,2022-01-24,2022-01-25,2022-01-25,5.0,5.0,2.0,1.0,100.0,100.0,100.0,100.0,21600.00,21600.00,10,No,4568323452_10,2022-01-04,us,100907802.0,san joaquin,ca
4,5004,us pl solon,100037407,nestrade s.a.,100364181,great lakes cheese,4567833735,R01,dairy products,43100075,cheese cheddar sharp accelerated,303,N,2021-11-12,2021-11-12,2021-11-12,2021-11-12,5.0,5.0,2.0,1.0,100.0,100.0,100.0,100.0,39999.96,39999.96,10,No,4567833735_10,2022-01-31,us,100920029.0,jefferson,ny


In [0]:
ves_finale.isnull().sum()

Out[88]: Plant_ID 0
Plant_Name 0
Vendor 0
VM_Name 0
Supplier_No. 0
VS_Name 0
Purchase_Order 0
Material_group_type 0
Material_Group 0
Material_No. 0
Material_Description 0
MRP_controller 0
Manual_Override 0
First_GR_Date 0
Last_GR_Date 0
First_Statistical_delivery_Date 0
Delivery_date 0
Under_Delievry_Tolerance_pct 0
Over_Delivery_Tolerance_pct 0
Early_Delivery_Tolerance_Days 0
Late_Delievry_Tolerance_Days 0
On-Time_Delivery_pct 0
Quantity_Reliability_pct 0
Initial_Quality_pct 0
Total_Quality_pct 0
Purchase_Order_Scheduled_Qty 0
Delivered_Quantity 0
PO_Item 0
Missed_PO 0
Purchase_order_PO_item 0
PO_Create_Date 0
updated_VS_country 0
updated_VS_ID 0
updated_VS_location 0
updated_VS_region 47
dtype: int64

In [0]:
#Splitting PO Qty and Unit of Measurement
#ves_finale[['Purchase_Order_Scheduled_Qty','Purchase_Order_Scheduled_Qty_UoM']]=ves_finale.Purchase_Order_Scheduled_Qty.str.split(' ',expand =True)
ves_finale['updated_VS_ID']=ves_finale['updated_VS_ID'].astype(np.int64)
ves_finale['Purchase_Order_Scheduled_Qty']=ves_finale['Purchase_Order_Scheduled_Qty'].astype(str)
d=list()
for i in ves_finale['Purchase_Order_Scheduled_Qty']:
  d.append(i.replace(',',''))
ves_finale['Purchase_Order_Scheduled_Qty']=d
ves_finale['Purchase_Order_Scheduled_Qty']=ves_finale['Purchase_Order_Scheduled_Qty'].astype(float)
ves_finale=ves_finale.drop_duplicates()

In [0]:
#2: Math for distance feature creation begins here
#Converting all character columns to lowercase
ves_finale['updated_VS_location'] = ves_finale['updated_VS_location'].str.lower()
ves_finale['updated_VS_region'] = ves_finale['updated_VS_region'].str.lower()
ves_finale['updated_VS_country'] = ves_finale['updated_VS_country'].str.lower()
ves_finale = ves_finale.reset_index()
ves_finale.drop(columns='index', inplace=True)

#Updating region columns for VS in North America and Outside North America
# ves_finale.loc[ves_finale['updated_VS_region'].str.split('/').str[0].isin(['us', 'mx', 'ca']), 
#                'updated_VS_region']=ves_finale['updated_VS_region'].str.split('/').str[1]
# ves_finale.loc[~ves_finale['updated_VS_region'].str.split('/').str[0].isin(['us', 'mx', 'ca']), 
#                'updated_VS_region']=ves_finale['updated_VS_region'].str.split('/').str[0]

# updating some countries and region values in the VES data
for vs_id, vs_reg in zip(ves_finale['updated_VS_location'],ves_finale['updated_VS_region']):
  if (((vs_id=='madison') | (vs_id=='payette')) & (vs_reg=='id')):
    ves_finale.loc[(ves_finale['updated_VS_location']==vs_id) & (ves_finale['updated_VS_region']==vs_reg),'updated_VS_country']='us'
  else:
    pass

ves_finale.loc[ves_finale['updated_VS_region']=='de','updated_VS_country']='de'
ves_finale.loc[ves_finale['updated_VS_region']=='br','updated_VS_country']='br'
ves_finale.loc[ves_finale['updated_VS_region']=='vn','updated_VS_country']='vt'
ves_finale.loc[ves_finale['updated_VS_country']=='vt','updated_VS_region']='vt'

In [0]:
#from records where distance is na subsetting records where location is not available
ves_location_na = ves_finale[(ves_finale['updated_VS_location'].isna()==True) | (ves_finale['updated_VS_location']=='#')]
#Removing records from QIR where VS no is missing
QIR_updated_data_v1 = QIR_updated_data[QIR_updated_data['VensitnoVS'].isna()==False]
QIR_updated_data_v1['VensitnoVS'] = QIR_updated_data_v1['VensitnoVS'].astype(int)
#Getting records from QIR where VS is present in VES VS for missing locations
ves_location_na_v1 = QIR_updated_data_v1[QIR_updated_data_v1['VensitnoVS'].isin(ves_location_na['updated_VS_ID'].unique().tolist())][['VensitnoVS', 
                                                                                                                                 'City(VS)', 
                                                                                                                                 'RG (VS)', 'Country VS']].drop_duplicates()

#Converting all character columns to lowercase
ves_location_na_v1['City(VS)'] = ves_location_na_v1['City(VS)'].str.lower()
ves_location_na_v1['RG (VS)'] = ves_location_na_v1['RG (VS)'].str.lower()
ves_location_na_v1['Country VS'] = ves_location_na_v1['Country VS'].str.lower()

#For VS locations missing in VES, extracting locations from QIR
for index, row in ves_location_na_v1.iterrows():
    ves_finale.loc[ves_finale['updated_VS_ID']==row['VensitnoVS'], 'updated_VS_location'] = row['City(VS)']
    ves_finale.loc[ves_finale['updated_VS_ID']==row['VensitnoVS'], 'updated_VS_region'] = row['RG (VS)']
    ves_finale.loc[ves_finale['updated_VS_ID']==row['VensitnoVS'], 'updated_VS_country'] = row['Country VS']

In [0]:
print(ves_finale.shape)
ves_finale.head(1)

(10598, 35)
Out[92]:

,Plant_ID,Plant_Name,Vendor,VM_Name,Supplier_No.,VS_Name,Purchase_Order,Material_group_type,Material_Group,Material_No.,Material_Description,MRP_controller,Manual_Override,First_GR_Date,Last_GR_Date,First_Statistical_delivery_Date,Delivery_date,Under_Delievry_Tolerance_pct,Over_Delivery_Tolerance_pct,Early_Delivery_Tolerance_Days,Late_Delievry_Tolerance_Days,On-Time_Delivery_pct,Quantity_Reliability_pct,Initial_Quality_pct,Total_Quality_pct,Purchase_Order_Scheduled_Qty,Delivered_Quantity,PO_Item,Missed_PO,Purchase_order_PO_item,PO_Create_Date,updated_VS_country,updated_VS_ID,updated_VS_location,updated_VS_region
0,5004,us pl solon,100037407,nestrade s.a.,100099158,leprino foods dairy products co,4567777068,R01,dairy products,22002012,cheese mozz prtsk pmsn flv shd iqf,303,N,2021-11-19,2021-11-19,2021-11-19,2021-11-19,5.0,5.0,2.0,1.0,100.0,100.0,100.0,100.0,19800.0,19800.0,10,No,4567777068_10,2022-01-04,us,100907802,san joaquin,ca


In [0]:
ves_finale.isnull().sum()

Out[93]: Plant_ID 0
Plant_Name 0
Vendor 0
VM_Name 0
Supplier_No. 0
VS_Name 0
Purchase_Order 0
Material_group_type 0
Material_Group 0
Material_No. 0
Material_Description 0
MRP_controller 0
Manual_Override 0
First_GR_Date 0
Last_GR_Date 0
First_Statistical_delivery_Date 0
Delivery_date 0
Under_Delievry_Tolerance_pct 0
Over_Delivery_Tolerance_pct 0
Early_Delivery_Tolerance_Days 0
Late_Delievry_Tolerance_Days 0
On-Time_Delivery_pct 0
Quantity_Reliability_pct 0
Initial_Quality_pct 0
Total_Quality_pct 0
Purchase_Order_Scheduled_Qty 0
Delivered_Quantity 0
PO_Item 0
Missed_PO 0
Purchase_order_PO_item 0
PO_Create_Date 0
updated_VS_country 0
updated_VS_ID 0
updated_VS_location 0
updated_VS_region 47
dtype: int64

In [0]:
# function to find the coordinates of a given city 
def findGeocode(city):   
    # try and catch is used to overcome the exception thrown by geolocator using geocodertimedout  
    try:
        # Specify the user_agent as your app name, it should not be none
        #Using the Photon function for geocoding
        geolocator = Photon(user_agent="myGeocoder")
        return geolocator.geocode(city)
    except GeocoderTimedOut:
        return findGeocode(city) 

In [0]:
#Creating VS location data for getting latitude and longitude
ves_data_for_lat_long = ves_finale[['updated_VS_ID', 'updated_VS_location', 'updated_VS_region']].drop_duplicates()
ves_data_for_lat_long_v1 = ves_data_for_lat_long[(ves_data_for_lat_long['updated_VS_location'].isna()==False) & 
                                                 (ves_data_for_lat_long['updated_VS_location']!='#') & 
                                                 (ves_data_for_lat_long['updated_VS_region'].isna()==False) & 
                                                 (ves_data_for_lat_long['updated_VS_region']!='30')]
ves_data_for_lat_long_v1['VS_city_reg'] = ves_data_for_lat_long_v1['updated_VS_location']+', '+ves_data_for_lat_long_v1['updated_VS_region']

#Initiating lists for storing VS latitude and longitude
longitude = []
latitude = []

#For loop for generating latitude and longitude for each VS
for i in (ves_data_for_lat_long_v1['VS_city_reg']):
  loc = findGeocode(i)
  if loc != None:
    # coordinates returned from function is stored into two separate list
    latitude.append(loc.latitude)
    longitude.append(loc.longitude)

    # if coordinate for a city not found, insert "NaN" indicating missing value 
  else:
    latitude.append(np.nan)
    longitude.append(np.nan)

#Adding latitude and longitude to VS location data
ves_data_for_lat_long_v1['vs_lat'] = latitude
ves_data_for_lat_long_v1['vs_lon'] = longitude

In [0]:
print(ves_finale.shape)
ves_finale.isnull().sum()

(10598, 35)
Out[96]: Plant_ID 0
Plant_Name 0
Vendor 0
VM_Name 0
Supplier_No. 0
VS_Name 0
Purchase_Order 0
Material_group_type 0
Material_Group 0
Material_No. 0
Material_Description 0
MRP_controller 0
Manual_Override 0
First_GR_Date 0
Last_GR_Date 0
First_Statistical_delivery_Date 0
Delivery_date 0
Under_Delievry_Tolerance_pct 0
Over_Delivery_Tolerance_pct 0
Early_Delivery_Tolerance_Days 0
Late_Delievry_Tolerance_Days 0
On-Time_Delivery_pct 0
Quantity_Reliability_pct 0
Initial_Quality_pct 0
Total_Quality_pct 0
Purchase_Order_Scheduled_Qty 0
Delivered_Quantity 0
PO_Item 0
Missed_PO 0
Purchase_order_PO_item 0
PO_Create_Date 0
updated_VS_country 0
updated_VS_ID 0
updated_VS_location 0
updated_VS_region 47
dtype: int64

In [0]:
#Creating Plant location data for getting latitude and longitude
plant_city_state['Plant ID'] = plant_city_state['Destination'].astype(str).str[0:4]
plant_city_state = plant_city_state[['Plant ID', 'd_CityState']]
plant_city_state = plant_city_state.drop_duplicates()
plant_city_state.rename(columns={'Plant ID':'Plant_ID'}, inplace=True)
plant_city_state['Plant_ID'] = plant_city_state['Plant_ID'].astype(int)
plant_city_state = plant_city_state[plant_city_state['Plant_ID'].isin(ves_finale['Plant_ID'].unique().tolist())]

#Initiating lists for storing plant latitude and longitude
plant_latitude = []
plant_longitude = []

#For loop for generating latitude and longitude for each VS
for i in (plant_city_state['d_CityState']):
  loc = findGeocode(i)
  if loc != None:
    # coordinates returned from function is stored into two separate list
    plant_latitude.append(loc.latitude)
    plant_longitude.append(loc.longitude)
       
    # if coordinate for a city not found, insert "NaN" indicating missing value 
  else:
    plant_latitude.append(np.nan)
    plant_longitude.append(np.nan)
    
#Adding latitude and longitude to Plant location data
plant_city_state['plant_lat'] = plant_latitude
plant_city_state['plant_lon'] = plant_longitude

In [0]:
#Merging VS lat long data and plant lat long data with VES data
ves_finale_v1 = ves_finale.merge(ves_data_for_lat_long_v1[['updated_VS_ID', 'vs_lat', 'vs_lon']], on='updated_VS_ID', how='left')
ves_finale_v2 = ves_finale_v1.merge(plant_city_state[['Plant_ID', 'plant_lat', 'plant_lon']], on='Plant_ID', how='left')

#Manually updating some VS latitudes and longitudes for missing values
ves_finale_v2.loc[ves_finale_v2['updated_VS_ID']==100937131, 'vs_lat'] = -34.84967
ves_finale_v2.loc[ves_finale_v2['updated_VS_ID']==100937131, 'vs_lon'] = 150.61093
ves_finale_v2.loc[ves_finale_v2['updated_VS_ID']==100184989, 'vs_lat'] = 38.0333
ves_finale_v2.loc[ves_finale_v2['updated_VS_ID']==100184989, 'vs_lon'] = -1.1167
ves_finale_v2.loc[ves_finale_v2['updated_VS_ID']==101162016, 'vs_lat'] = 10.964112
ves_finale_v2.loc[ves_finale_v2['updated_VS_ID']==101162016, 'vs_lon'] = 106.856461

In [0]:
#Creating haversine function for calculating bird distance for VS outside North America
def haversine(row):
    lon1 = row['vs_lon']
    lat1 = row['vs_lat']
    lon2 = row['plant_lon']
    lat2 = row['plant_lat']
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * arcsin(sqrt(a)) 
    km = 6367 * c
    return km

In [0]:
#Creating all the unique PLANT VS combos from VES data
plant_vs_combo_ves = ves_finale_v2[['Plant_ID', 'updated_VS_ID', 'updated_VS_location', 'updated_VS_region', 'updated_VS_country', 
                                    'vs_lat', 'vs_lon', 'plant_lat', 'plant_lon']].drop_duplicates()

#Filtering out records where VS location is not available
plant_vs_combo_ves = plant_vs_combo_ves[plant_vs_combo_ves['vs_lat'].isna()==False]

#Splitting Plant VS combos between North America and outside North America
distance_north_america = plant_vs_combo_ves[plant_vs_combo_ves['updated_VS_country'].isin(['us', 'mx', 'ca'])]
distance_file_iternational = plant_vs_combo_ves[~plant_vs_combo_ves['updated_VS_country'].isin(['us', 'mx', 'ca'])]

#Creating distance column with initial values as 0
distance_north_america['distance'] = 0
distance_north_america = distance_north_america.reset_index()
distance_north_america.drop(columns=['index'], inplace=True)

#For VS within North America, using OSRM package to calculate distance between Plant and VS
for index, row in distance_north_america.iterrows():
    print(index)
    r = requests.get(f"http://router.project-osrm.org/route/v1/car/{row['vs_lon']},{row['vs_lat']};{row['plant_lon']},{row['plant_lat']}?overview=false""")
    routes = json.loads(r.content)
    if 'routes' not in list(routes.keys()):
      distance=0
    else:
      distance = routes.get('routes')[0].get('distance')
    print(distance)
    distance_north_america.iloc[index, distance_north_america.columns.get_loc('distance')] = distance

#Diving distance calculated using OSRM package by 1000 to get distance in KMs
distance_north_america['distance'] = distance_north_america['distance']/1000

#Applying haversine formula to get distance between PLant and VS outside north America
distance_file_iternational['distance'] = distance_file_iternational.apply(lambda row: haversine(row), axis=1)

#Combining all the distances in a single dataframe
distance_v1 = pd.concat([distance_north_america, distance_file_iternational], ignore_index=True)

0
3913994.3
1
716444.5
2
1291629.7
3
525637.7
4
800278.4
5
800278.4
6
271035.9
7
124976
8
1129644.4
9
1136163.1
10
944201.7
11
648661.2
12
3913994.3
13
4023693
14
1282900.5
15
1084385.2
16
1022856.5
17
819990.4
18
428656.2
19
289969.9
20
3726441.9
21
1252938.6
22
705807.5
23
719720.6
24
1593180.3
25
925985.9
26
1090941.9
27
842502.8
28
898648.3
29
826636
30
452111.1
31
2727000
32
713353.2
33
2018314.1
34
327806.7
35
230707
36
208037.6
37
579570
38
967913.6
39
495499.4
40
1207019.3
41
298980
42
825959.7
43
686036.6
44
1207019.3
45
1291926.9
46
1778041.1
47
1863083.8
48
811377
49
1984308.5
50
318096.2
51
540810.8
52
25273.6
53
248070
54
377842.3
55
1042857.9
56
819514.5
57
721057.6
58
215982.4
59
0
60
1069728.2
61
1042857.9
62
2430142.6
63
1598875.1
64
131521.5
65
531060.9
66
1065612
67
1584557.8
68
1126739.8
69
704772.5
70
2184483.7
71
531060.9
72
4809554.9
73
2576532
74
471786
75
1071425.6
76
564569.8
77
409879
78
812575.9
79
468131.4
80
1029749.3
81
1560323
82
780658.3
83
3462315.9
84
1287559.3
85
14075280.6
86
562734.3
87
1463215.5
88
1940825.8
89
381746.2
90
606984.8
91
1673856.6
92
3157882.6
93
3204927.1
94
1696893.5
95
575208.2
96
687834.3
97
1052180.4
98
784754.4
99
1189985.3
100
891453.9
101
3413108.8
102
3153950.8
103
669763.9
104
139754.1
105
2177265
106
7300.1
107
1854608
108
1227368.8
109
367003.6
110
539449.5
111
2438422.2
112
2749481.7
113
4044372.7
114
1339455.7
115
0
116
1416514.4
117
145197.1
118
1716354.3
119
1773268.5
120
1329410.8
121
2009705.5
122
2015005.2
123
1107651.7
124
616056.5
125
1572583.5
126
1741195.3
127
409670.7
128
588037.6
129
2873839.1
130
1091057.9
131
528041
132
2002597.4
133
1110066.6
134
4345351.6
135
1629301.8
136
1443600.1
137
1288937.4
138
602608.4
139
14075276.2
140
1284067.5
141
1165214.2
142
1149926
143
1496108.9
144
3512702.7
145
610459.3
146
1277179.9
147
1209719.9
148
2665985.7
149
1187136.2
150
1192632.6
151
2809772.7
152
336376.3
153
3544521.1
154
206534.9
155
2836567.6
156
1730359.1
157
2437295.7
158
132527.8
159
2610801.4
160
2033973.8
161
2040789
162
82542.8
163
2242858
164
1937890.1
165
2149539.4
166
1302331.3
167
2974713.3
168
132527.8
169
1302331.3
170
2278335.4
171
2345172.9
172
82542.8
173
2277210.9
174
143644.7
175
2132841.1
176
2286329.5
177
2318015.5
178
3579206
179
2167505.6
180
2659248.5
181
82542.8
182
2694074.7
183
2692044.6
184
782129.1
185
126064.7
186
1546348.6
187
31535.9
188
403597.6
189
662367.7
190
479632.5
191
1210710.7
192
3269032.8
193
565885.4
194
3584118.8
195
3788961.1
196
2160015.7
197
581016.8
198
275109.6
199
0
200
587593
201
629947.3
202
327806.7
203
510561.1
204
1341815.6
205
1272362.7
206
510561.1
207
1054624.1
208
2792067.8
209
32325.3
210
1319540.2
211
3341187.8
212
1265984.7
213
861140.8
214
1499901.1
215
1498940
216
852164
217
1381088.8
218
3848041.3
219
1498522.6
220
296971.8
221
373726.7
222
1646360.3
223
1219521.8
224
1197009.2
225
1052180.4
226
173860.4
227
996384.4
228
1572728.2
229
1200429.3
230
826497.9
231
1708008.9
232
2837406.2
233
1533133.5
234
1826187.2
235
173860.4
236
814411.2
237
1200429.3
238
458715.4
239
1548836.5
240
3319179.8
241
367003.6
242
14075280.6
243
778918.1
244
481360
245
1147493.4
246
0
247
1757449.5
248
1524691.1
249
1656833.8
250
1161288.4
251
1264694.5
252
4465697
253
3890313.7
254
1748008.6
255
1210706.4
256
3887942.4
257
1554006.9
258
2212123.6
259
1110066.6
260
4393927.3
261
3889836.1
262
2650852.9
263
2075272.7
264
1966654.9
265
1554006.9
266
1686343.2
267
1057743.6
268
2863274.5
269
2690148.9
270
2401697
271
3098852.9
272
1210710.7
273
621569.3
274
2278335.4
275
3385766.7
276
1078978.5
277
1127554.2
278
0
279
1032599.6
280
765209.7
281
2278335.4
282
0
283
2350291.9
284
4159389.4
285
3407474.3
286
3449828.6
287
1778612.8

In [0]:
#Manually calculating some distances for missing values
distance_v1.loc[(distance_v1['Plant_ID']==5004) & (distance_v1['updated_VS_ID']==100663389), 'distance'] = 1315.9606
distance_v1.loc[(distance_v1['Plant_ID']==5878) & (distance_v1['updated_VS_ID']==100663389), 'distance'] = 644.38134
distance_v1.loc[(distance_v1['Plant_ID']==5955) & (distance_v1['updated_VS_ID']==100663389), 'distance'] = 1506.9897
distance_v1.loc[(distance_v1['Plant_ID']==5959) & (distance_v1['updated_VS_ID']==100663389), 'distance'] = 1759.978598
distance_v1.loc[(distance_v1['Plant_ID']==5259) & (distance_v1['updated_VS_ID']==100855588), 'distance'] = 811.75311
distance_v1.loc[(distance_v1['Plant_ID']==5004) & (distance_v1['updated_VS_ID']==100358246), 'distance'] = 669.96991
distance_v1.loc[(distance_v1['Plant_ID']==5004) & (distance_v1['updated_VS_ID']==100874123), 'distance'] = 979.92956
distance_v1.loc[(distance_v1['Plant_ID']==5878) & (distance_v1['updated_VS_ID']==100874123), 'distance'] = 1201.2144
distance_v1.loc[(distance_v1['Plant_ID']==5955) & (distance_v1['updated_VS_ID']==100351791), 'distance'] = 237.70011
distance_v1.loc[(distance_v1['Plant_ID']==5955) & (distance_v1['updated_VS_ID']==100358246), 'distance'] = 917.64795
distance_v1.loc[(distance_v1['Plant_ID']==5955) & (distance_v1['updated_VS_ID']==100874123), 'distance'] = 218.54892
distance_v1.loc[(distance_v1['Plant_ID']==5959) & (distance_v1['updated_VS_ID']==100351791), 'distance'] = 3085.91712
distance_v1.loc[(distance_v1['Plant_ID']==5959) & (distance_v1['updated_VS_ID']==100358246), 'distance'] = 3488.41405
distance_v1.loc[(distance_v1['Plant_ID']==5959) & (distance_v1['updated_VS_ID']==100838018), 'distance'] = 7745.1141066568825
distance_v1.loc[(distance_v1['Plant_ID']==5959) & (distance_v1['updated_VS_ID']==100874123), 'distance'] = 3443.03055

#Subsetting Plant_ID, VS ID and distance columns to merge with VES data
distance_v1 = distance_v1[['Plant_ID', 'updated_VS_ID', 'distance']].drop_duplicates()

#Merging distance data with VES data on Plant and VS ID
ves_finale_v3 = ves_finale_v2.merge(distance_v1[['Plant_ID', 'updated_VS_ID', 'distance']], on=['Plant_ID', 'updated_VS_ID'], how='left')

#Creating domestic/international flag
ves_finale_v3.loc[ves_finale_v3['updated_VS_country']=='us', 'dom_or_int'] = 'dom'
ves_finale_v3.loc[ves_finale_v3['updated_VS_country']!='us', 'dom_or_int'] = 'int'

In [0]:
print(ves_finale_v3.shape)
ves_finale_v3.isnull().sum()

(10598, 41)
Out[102]: Plant_ID 0
Plant_Name 0
Vendor 0
VM_Name 0
Supplier_No. 0
VS_Name 0
Purchase_Order 0
Material_group_type 0
Material_Group 0
Material_No. 0
Material_Description 0
MRP_controller 0
Manual_Override 0
First_GR_Date 0
Last_GR_Date 0
First_Statistical_delivery_Date 0
Delivery_date 0
Under_Delievry_Tolerance_pct 0
Over_Delivery_Tolerance_pct 0
Early_Delivery_Tolerance_Days 0
Late_Delievry_Tolerance_Days 0
On-Time_Delivery_pct 0
Quantity_Reliability_pct 0
Initial_Quality_pct 0
Total_Quality_pct 0
Purchase_Order_Scheduled_Qty 0
Delivered_Quantity 0
PO_Item 0
Missed_PO 0
Purchase_order_PO_item 0
PO_Create_Date 0
updated_VS_country 0
updated_VS_ID 0
updated_VS_location 0
updated_VS_region 47
vs_lat 0
vs_lon 0
plant_lat 0
plant_lon 0
distance 0
dom_or_int 0
dtype: int64

In [0]:
# ves_finale_v3[ves_finale_v3['updated_VS_country'].isnull()].shape

In [0]:
incoterms_data = incoterms_data[incoterms_data['Header Incoterms']=='EXW']
incoterms_data.rename(columns={'Header Incoterms':'header_incoterms'}, inplace=True)
incoterms_data = incoterms_data.drop_duplicates()
ves_finale_v4 = ves_finale_v3.merge(incoterms_data, on=['Plant_ID', 'Vendor', 'Material_No.'], how='left')
ves_finale_v4.loc[ves_finale_v4['header_incoterms']=='EXW', 'nestle_managed_freight'] = 'yes'
ves_finale_v4.loc[ves_finale_v4['header_incoterms'].isna()==True, 'nestle_managed_freight'] = 'no'

In [0]:
ves_finale_v4['Total_ordered_QTY'] = ves_finale_v4.groupby(by=['Plant_ID','Material_No.'])['Purchase_Order_Scheduled_Qty'].transform(np.sum)
ves_finale_v4['Total_Purchase_ordered_QTY'] = ves_finale_v4.groupby(by=['Plant_ID','Material_No.','updated_VS_ID'])['Purchase_Order_Scheduled_Qty'].transform(np.sum)
ves_finale_v4['vendor_percentage']=round((ves_finale_v4['Total_Purchase_ordered_QTY']/ves_finale_v4['Total_ordered_QTY'])*100,2)
mat_count=[]
for p_id,vs_id in zip(ves_finale_v4['Plant_ID'],ves_finale_v4['updated_VS_ID']):
  a=ves_finale_v4[(ves_finale_v4['Plant_ID']==p_id) & (ves_finale_v4['updated_VS_ID']==vs_id)]
  c=len(a['Material_No.'].unique())
  mat_count.append(c)
ves_finale_v4['vendor_material_cnt_per_plant']=mat_count

In [0]:
bom1=pd.read_csv('enter file path to BOM_5004-1.csv')#Put it at the top
bom2=pd.read_csv('enter file path to BOM_V1_wo5004_v1.csv')#Put it at the top
bom = pd.concat([bom1, bom2], ignore_index=True)
bom.drop(bom[bom['ComponentMaterialNumber'].isnull()].index,axis=0,inplace=True)
bom['ComponentMaterialNumber']=bom['ComponentMaterialNumber'].astype(np.int64)
bom=bom[['LocationID','BillOfMaterial','ComponentMaterialNumber']]
bom.drop_duplicates(inplace=True)
bom_cont1=pd.pivot_table(bom,values=['BillOfMaterial'], index=['LocationID','ComponentMaterialNumber'],aggfunc='count').reset_index()
bom_cont1.rename(columns={'LocationID':'Plant_ID','ComponentMaterialNumber':'Material_No.'},inplace=True)
ves_finale_v5=pd.merge(ves_finale_v4,bom_cont1,on=['Plant_ID','Material_No.'],how='left')
ves_finale_v5.drop(ves_finale_v5[ves_finale_v5['BillOfMaterial'].isnull()].index,inplace=True)
ves_finale_v5['sku_type']=['single' if i<=1 else 'multiple' for i in ves_finale_v5['BillOfMaterial']]

In [0]:
ves_finale_v5['Material_group_type'].str[0].unique()

Out[107]: array(['R', 'P'], dtype=object)

In [0]:
ves_finale_v5['mode_of_transport'] = 'road'
ves_finale_v5.loc[(ves_finale_v5['Plant_ID'].isin([5258, 5259, 5446])) & 
                             (ves_finale_v5['Material_No.'].isin([22001236, 22002545, 22019000, 43433857])), 
                             'mode_of_transport'] = 'rail'
ves_finale_v5['Material_group_type'] = ves_finale_v5['Material_group_type'].str[0]

In [0]:
ves_finale_v5['PO_Create_Month'] = ves_finale_v5['PO_Create_Date'].dt.month
ves_finale_v5.loc[ves_finale_v5['vendor_percentage']==100, 'single_or_multisource'] = 'single'
ves_finale_v5.loc[ves_finale_v5['vendor_percentage']!=100, 'single_or_multisource'] = 'multi'

In [0]:
ves_finale_v5 = pd.get_dummies(ves_finale_v5, columns=['sku_type', 'dom_or_int', 'Material_group_type', 'nestle_managed_freight', 
                                                       'mode_of_transport', 'single_or_multisource'],drop_first=True)
ves_finale_v5 = pd.get_dummies(ves_finale_v5, columns=['PO_Create_Month'])

In [0]:
us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())
def bus_day_count(day1, day2):
    return(len(pd.date_range(day1, day2, freq=us_bd).tolist()))
#Applying the function
ves_finale_v5['bus_day_count'] = ves_finale_v5.apply(lambda x: bus_day_count(x['PO_Create_Date'], x['Delivery_date']), axis=1)
#Calculating total days (lead time including weekends and holidays) from PO Create Date to Delivery Date
ves_finale_v5['total_days'] = (pd.to_datetime(ves_finale_v5['Delivery_date']) - pd.to_datetime(ves_finale_v5['PO_Create_Date'])).dt.days
#Calculation holiday as difference of from total days and business days
ves_finale_v5['holiday_weekend_count'] = ves_finale_v5['total_days']-ves_finale_v5['bus_day_count']

ves_finale_v5.loc[(ves_finale_v5['Quantity_Reliability_pct']=='100') & 
                  (ves_finale_v5['On-Time_Delivery_pct']=='100') & 
                  (ves_finale_v5['Total_Quality_pct']=='100'), 'OTIF'] = 1
ves_finale_v5.loc[ves_finale_v5['OTIF'].isna()==True, 'OTIF'] = 0

In [0]:
print(ves_finale_v5.shape)
ves_finale_v5.isnull().sum()

(10185, 66)
Out[112]: Plant_ID 0
Plant_Name 0
Vendor 0
VM_Name 0
Supplier_No. 0
VS_Name 0
Purchase_Order 0
Material_Group 0
Material_No. 0
Material_Description 0
MRP_controller 0
Manual_Override 0
First_GR_Date 0
Last_GR_Date 0
First_Statistical_delivery_Date 0
Delivery_date 0
Under_Delievry_Tolerance_pct 0
Over_Delivery_Tolerance_pct 0
Early_Delivery_Tolerance_Days 0
Late_Delievry_Tolerance_Days 0
On-Time_Delivery_pct 0
Quantity_Reliability_pct 0
Initial_Quality_pct 0
Total_Quality_pct 0
Purchase_Order_Scheduled_Qty 0
Delivered_Quantity 0
PO_Item 0
Missed_PO 0
Purchase_order_PO_item 0
PO_Create_Date 0
updated_VS_country 0
updated_VS_ID 0
updated_VS_location 0
updated_VS_region 47
vs_lat 0
vs_lon 0
plant_lat 0
plant_lon 0
distance 0
header_incoterms 8699
Total_ordered_QTY 0
Total_Purchase_ordered_QTY 0
vendor_percentage 0
vendor_material_cnt_per_plant 0
BillOfMaterial 0
sku_type_single 0
dom_or_int_int 0
Material_group_type_R 0
nestle_managed_freight_yes 0
mode_of_transport_road 0
single_or_multisource_single 0
PO_Create_Month_1 0
PO_Create_Month_2 0
PO_Create_Month_3 0
PO_Create_Month_4 0
PO_Create_Month_6 0
PO_Create_Month_7 0
PO_Create_Month_8 0
PO_Create_Month_9 0
PO_Create_Month_10 0
PO_Create_Month_11 0
PO_Create_Month_12 0
bus_day_count 0
total_days 0
holiday_weekend_count 0
OTIF 0
dtype: int64

In [0]:
#Renaming total_days to lead_time
ves_finale_v5.rename(columns={'total_days':'lead_time'}, inplace=True)

#Dropping columns not required in the model
ves_finale_v5.drop(columns=['Supplier_No.', 'VS_Name', 'BillOfMaterial', 'vs_lon', 'vs_lat', 'plant_lon', 'plant_lat', 'Total_ordered_QTY', 
                            'Total_Purchase_ordered_QTY', 'On-Time_Delivery_pct', 'Quantity_Reliability_pct', 'Initial_Quality_pct', 
                            'Total_Quality_pct', 'Under_Delievry_Tolerance_pct', 'Over_Delivery_Tolerance_pct', 
                            'Early_Delivery_Tolerance_Days', 'Late_Delievry_Tolerance_Days', 'MRP_controller', 
                            'Manual_Override', 'header_incoterms'], inplace=True)

#Dropping roecords where distance in na because VS location is not available
ves_finale_v5 = ves_finale_v5.dropna(subset=['distance'])

#Dropping roecords where PO_Create_Date is not available
ves_finale_v5 = ves_finale_v5[ves_finale_v5['PO_Create_Date']!='1888-01-01']

#Converting region column to str datatype to write to spark table
ves_finale_v5['updated_VS_region'] = ves_finale_v5['updated_VS_region'].astype(str)

In [0]:
print(ves_finale_v5.shape)
ves_finale_v5.isnull().sum()

(10097, 46)
Out[114]: Plant_ID 0
Plant_Name 0
Vendor 0
VM_Name 0
Purchase_Order 0
Material_Group 0
Material_No. 0
Material_Description 0
First_GR_Date 0
Last_GR_Date 0
First_Statistical_delivery_Date 0
Delivery_date 0
Purchase_Order_Scheduled_Qty 0
Delivered_Quantity 0
PO_Item 0
Missed_PO 0
Purchase_order_PO_item 0
PO_Create_Date 0
updated_VS_country 0
updated_VS_ID 0
updated_VS_location 0
updated_VS_region 0
distance 0
vendor_percentage 0
vendor_material_cnt_per_plant 0
sku_type_single 0
dom_or_int_int 0
Material_group_type_R 0
nestle_managed_freight_yes 0
mode_of_transport_road 0
single_or_multisource_single 0
PO_Create_Month_1 0
PO_Create_Month_2 0
PO_Create_Month_3 0
PO_Create_Month_4 0
PO_Create_Month_6 0
PO_Create_Month_7 0
PO_Create_Month_8 0
PO_Create_Month_9 0
PO_Create_Month_10 0
PO_Create_Month_11 0
PO_Create_Month_12 0
bus_day_count 0
lead_time 0
holiday_weekend_count 0
OTIF 0
dtype: int64

In [0]:
#null value treatment
print('shape before null treatment: ',smi_1.shape)
smi_1.drop(smi_1[smi_1['Vendor Number'].isnull()].index,inplace=True)
smi_1['Material Number']=smi_1['Material Number'].astype(np.int64)
smi_1['Vendor Number']=smi_1['Vendor Number'].astype(np.int64)
smi_1.drop(smi_1[smi_1['Vendor Site'].isnull()].index,inplace=True)
smi_1['Vendor Site']=smi_1['Vendor Site'].astype(np.int64)
smi_1.rename(columns={'Vendor Number':'Vendor','Vendor Site':'updated_VS_ID','PO Number':'Purchase_Order','Material Number':'Material_No.','Material Order Quantity':'Purchase_Order_Scheduled_Qty'},inplace=True)
print('shape after null treatment: ',smi_1.shape)

shape before null treatment: (5764, 35)
shape after null treatment: (5718, 35)

In [0]:
ves_f1=ves_finale_v5.copy()

In [0]:
#ves & SMI datatypes defined
ves_f1['Vendor']=ves_f1['Vendor'].astype(np.int64)
ves_f1['updated_VS_ID']=ves_f1['updated_VS_ID'].astype(np.int64)
ves_f1['Material_No.']=ves_f1['Material_No.'].astype(np.int64)
ves_f1['Purchase_Order_Scheduled_Qty']=ves_f1['Purchase_Order_Scheduled_Qty'].astype(float)
smi_1['Vendor']=smi_1['Vendor'].astype(np.int64)
smi_1['updated_VS_ID']=smi_1['updated_VS_ID'].astype(np.int64)
smi_1['Material_No.']=smi_1['Material_No.'].astype(np.int64)
smi_1['Purchase_Order_Scheduled_Qty']=smi_1['Purchase_Order_Scheduled_Qty'].astype(float)

In [0]:
# checking for same PO and material more than one complaint are registered with different complaint ID.
smi_1[(smi_1['Purchase_Order']==4567367243) & (smi_1['Material_No.']==44092721)]

Out[118]:

,Material Group Type,Month,Year,Division,Complaint ID,Current Stage,Status,Purchase_Order,Incident Date,Date Created,Location,Created By,Email,Vendor,Vendor Name1,updated_VS_ID,Supplier to Assign to,City,Region,Country,Material Group,Material Group Text,Material_No.,Material Description,Purchase_Order_Scheduled_Qty,Complain Category,Complaint Types,Complaint Description,Issue Description,Lot Disposition,Impact to Business,Supplier Response,Supplier Response Date,Due Date,Globe/Resource Key (Line Number)
1,Packaging,Oct,2021,Pizza/ Snacking,46886,NaN,Closed,4567367243,2021-10-09,2021-10-09,Mt Sterling,Ivy Planck,dustin.yahn@gapac.com,100365239,GP Operations Holdings LLC,101252654,Supplier Site Georgia-Pacific Corporation - Ci...,Circleville,OH,US,P20,Corrugated,44092721,CaseCorrugated Generic 12 Pack 2x2 10/20,28800.0,NFS,Packaging,Corrugate Printing - Color Variation,A pallet of material 44092721 has issue on one...,NaN,Minor - Routine Manufacturing Inconvenience,NaN,NaT,NaT,NaN
2,Packaging,Oct,2021,Pizza/ Snacking,46885,NaN,Closed,4567367243,2021-10-09,2021-10-09,Mt Sterling,Ivy Planck,dustin.yahn@gapac.com,100365239,GP Operations Holdings LLC,101252654,Supplier Site Georgia-Pacific Corporation - Ci...,Circleville,OH,US,P20,Corrugated,44092721,CaseCorrugated Generic 12 Pack 2x2 10/20,28800.0,NFS,Packaging,Corrugate Printing - Color Variation,A pallet of material 44092721 has issue on one...,NaN,Minor - Routine Manufacturing Inconvenience,NaN,NaT,NaT,NaN


In [0]:
# feature creation for Impact types
a=list()
for i in smi_1['Impact to Business']:
  a.append(i.split(' - ')[0])

smi_1['Impact types']=a

In [0]:
# feature creation for last 3m date & last 12m date
date_len=[90,365]
date_attr=['l3m_date','l12m_date']
for i,j in zip(date_len,date_attr):
  td = timedelta(i)
  ves_f1[j]=ves_f1['PO_Create_Date']-td

In [0]:
# feature creation for number of l3m & l12m complaints
date_attr=['l3m_date','l12m_date']
mnth=[3,12]
for date_att,mth in zip(date_attr,mnth):
  d='no.of.complaint_{}_months'.format(mth)
  col_str=[]
  for vs_id,mn,end_date,start_date in zip(ves_f1['updated_VS_ID'],ves_f1['Material_No.'],ves_f1['PO_Create_Date'],ves_f1[date_att]):
    a=smi_1[(smi_1['updated_VS_ID']==vs_id) & (smi_1['Material_No.']==mn) & (smi_1['Incident Date']<=end_date) & (smi_1['Incident Date']>=start_date)]
    col_str.append(a['updated_VS_ID'].count())
  ves_f1[d]=col_str

In [0]:
# feature creation for number of l3m complaints major & minor
for types in smi_1['Impact types'].unique().tolist():
  d='no.of.complaint_3_months_{}'.format(types.lower())
  col_str=[]
  for vs_id,mn,end_date,start_date in zip(ves_f1['updated_VS_ID'],ves_f1['Material_No.'],ves_f1['PO_Create_Date'],ves_f1['l3m_date']):
    a=smi_1[(smi_1['updated_VS_ID']==vs_id) & (smi_1['Material_No.']==mn) & (smi_1['Incident Date']<=end_date) & (smi_1['Incident Date']>=start_date)]
    b=a[a['Impact types']==types]
    col_str.append(b['updated_VS_ID'].count())
    
  ves_f1[d]=col_str

In [0]:
#feature creation for number of l12m complaints major & minor
for types in smi_1['Impact types'].unique().tolist():
  d='no.of.complaint_12_months_{}'.format(types.lower())
  col_str=[]
  for vs_id,mn,end_date,start_date in zip(ves_f1['updated_VS_ID'],ves_f1['Material_No.'],ves_f1['PO_Create_Date'],ves_f1['l12m_date']):
    a=smi_1[(smi_1['updated_VS_ID']==vs_id) & (smi_1['Material_No.']==mn) & (smi_1['Incident Date']<=end_date) & (smi_1['Incident Date']>=start_date)]
    b=a[a['Impact types']==types]
    col_str.append(b['updated_VS_ID'].count())
    
  ves_f1[d]=col_str

In [0]:
#feature creation for l3m complaints open
l3m_open=[]
for vs_id,mn,end_date,start_date in zip(ves_f1['updated_VS_ID'],ves_f1['Material_No.'],ves_f1['PO_Create_Date'],ves_f1['l3m_date']):
  a=smi_1[(smi_1['updated_VS_ID']==vs_id) & (smi_1['Material_No.']==mn) & (smi_1['Incident Date']<=end_date) & (smi_1['Incident Date']>=start_date)]
  b=a[a['Status']=="Open"]
  l3m_open.append(b['updated_VS_ID'].count())
  
ves_f1['no.of.complaint_3_months_open']=l3m_open

In [0]:
#feature creation for number of l3m complaints open major & minor
for types in smi_1['Impact types'].unique().tolist():
  d='no.of.complaint_3_months_open_{}'.format(types.lower())
  col_str=[]
  for vs_id,mn,end_date,start_date in zip(ves_f1['updated_VS_ID'],ves_f1['Material_No.'],ves_f1['PO_Create_Date'],ves_f1['l3m_date']):
    a=smi_1[(smi_1['updated_VS_ID']==vs_id) & (smi_1['Material_No.']==mn) & (smi_1['Incident Date']<=end_date) & (smi_1['Incident Date']>=start_date)]
    b=a[a['Status']=="Open"]
    c=b[b['Impact types']==types]
    col_str.append(c['updated_VS_ID'].count())
    
  ves_f1[d]=col_str

In [0]:
#feature creation for l12m complaint types
for cmp_typ in smi_1['Complaint Types'].unique().tolist():
  d='no.of.Complaint_type_{}_l12m'.format(cmp_typ)
  g='Complaint_type_{}_l12m_distinct'.format(cmp_typ)
  Complaint_type=[]
  for vs_id,mn,end_date,start_date in zip(ves_f1['updated_VS_ID'],ves_f1['Material_No.'],ves_f1['PO_Create_Date'],ves_f1['l12m_date']):
    a=smi_1[(smi_1['updated_VS_ID']==vs_id) & (smi_1['Material_No.']==mn) & (smi_1['Incident Date']<=end_date) & (smi_1['Incident Date']>=start_date)]
    b=a[a['Complaint Types']==cmp_typ]
    Complaint_type.append(b['updated_VS_ID'].count())

  ves_f1[d]=Complaint_type
  ves_f1[g]=[1 if i!=0 else 0 for i in ves_f1[d]]
  
cols=['_'.join(i.split(' ')) for i in ves_f1.columns.tolist()]
ves_f1.columns = cols

In [0]:
ves_smi=ves_f1.copy()
ves_smi.shape

Out[127]: (10097, 71)

In [0]:
#null value treatment
fin.drop(fin[fin['Risk Level'].isnull()].index,inplace=True)

#getting dummies for catg variables
fin_dat=pd.get_dummies(fin,columns=['Risk Level','Rating'])
fin_dat.drop(['Supplier'],axis=1,inplace=True)
fin_dat.rename(columns={'Supplier Number':'Vendor'},inplace=True)

#ves_smi data merged with financials
ves_smi_fin=pd.merge(ves_smi,fin_dat,on=['Vendor'],how='left')

#null treatment
ves_smi_fin.fillna(0,inplace=True)

cols=['_'.join(i.split(' ')) for i in ves_smi_fin.columns.tolist()]
ves_smi_fin.columns = cols

In [0]:
#selecting the required fields and sorting the data in ascending order
ves_smi_fin_cut=ves_smi_fin[['Plant_ID','Material_No.','updated_VS_ID','PO_Create_Date']]
ves_smi_fin_cut=ves_smi_fin_cut.sort_values(by=['Plant_ID','Material_No.','updated_VS_ID','PO_Create_Date'])
ves_smi_fin_cut.reset_index(inplace=True)
ves_smi_fin_cut.reset_index(inplace=True)
ves_smi_fin_cut.head(5)

Out[129]:

,level_0,index,Plant_ID,Material_No.,updated_VS_ID,PO_Create_Date
0,0,688,5004,22000848,100885237,2021-09-01
1,1,689,5004,22000848,100885237,2021-10-13
2,2,690,5004,22000848,100885237,2021-10-13
3,3,691,5004,22000848,100885237,2021-11-10
4,4,692,5004,22000848,100885237,2021-11-10


In [0]:
# function to obtain the days gap between the consecutive orders
def checkPrevious(x):
  try:
    if(x.level_0 - 1 >= 0 and x.updated_VS_ID == ves_smi_fin_cut.loc[x.level_0-1, 'updated_VS_ID'] and 
       x.Plant_ID == ves_smi_fin_cut.loc[x.level_0-1,'Plant_ID'] and x['Material_No.'] == ves_smi_fin_cut.loc[x.level_0-1, 'Material_No.']):
      
      return (x.PO_Create_Date - ves_smi_fin_cut.loc[x.level_0 -1, 'PO_Create_Date']).days
    return 0
  except:
    print("Exception")
    return 0
ves_smi_fin_cut['days']= ves_smi_fin_cut.apply(checkPrevious, 1)
ves_smi_fin_cut.head(5)

Out[130]:

,level_0,index,Plant_ID,Material_No.,updated_VS_ID,PO_Create_Date,days
0,0,688,5004,22000848,100885237,2021-09-01,0
1,1,689,5004,22000848,100885237,2021-10-13,42
2,2,690,5004,22000848,100885237,2021-10-13,0
3,3,691,5004,22000848,100885237,2021-11-10,28
4,4,692,5004,22000848,100885237,2021-11-10,0


In [0]:
# average of po create days difference(order frequency)
days_non_zero=ves_smi_fin_cut[ves_smi_fin_cut['days']>0]
days_non_zero_avg=pd.pivot_table(days_non_zero, values = 'days', index=['Plant_ID','Material_No.','updated_VS_ID'],aggfunc='mean').reset_index()
days_zero=ves_smi_fin_cut[ves_smi_fin_cut['days']==0]
days_zero_uni=days_zero[['Plant_ID','Material_No.','updated_VS_ID']].drop_duplicates()
order_freq=pd.merge(days_zero_uni,days_non_zero_avg, how='left', on = ['Plant_ID','Material_No.','updated_VS_ID'])
order_freq=order_freq.fillna(0)
order_freq.head(5)

Out[131]:

,Plant_ID,Material_No.,updated_VS_ID,days
0,5004,22000848,100885237,22.500000
1,5004,22000873,101136833,20.000000
2,5004,22001170,100364230,37.500000
3,5004,22001170,100366340,21.250000
4,5004,22001236,100735436,12.090909


In [0]:
#Order Freq Definitions: 0-6 days: Daily;, 7-20 days:Weekly; 21-40 days: Monthly, 41-100 days: Quarterly, 101-200 days: Half Yearly, 0 or more than 200 days: Yearly
#converting the average order_freq as per the condition and merge with the original table
conditions = [
    (order_freq['days'] > 0) & (order_freq['days'] <= 6),
    (order_freq['days'] > 6) & (order_freq['days'] <= 20),
    (order_freq['days'] > 20) & (order_freq['days'] <= 40),
    (order_freq['days'] > 40) & (order_freq['days'] <= 100),
    (order_freq['days'] > 100) & (order_freq['days'] <= 200),
    (order_freq['days'] > 200) | (order_freq['days']== 0)
    ]

values = ['Daily', 'Weekly', 'Monthly', 'Quarterly','Half Yearly','Yearly']
order_freq['Delivery_freq'] = np.select(conditions, values)
ves_smi_fin_freq = pd.merge(ves_smi_fin,order_freq, how='left', on = ['Plant_ID','Material_No.','updated_VS_ID'])
order_freq.head(5)

Out[132]:

,Plant_ID,Material_No.,updated_VS_ID,days,Delivery_freq
0,5004,22000848,100885237,22.500000,Monthly
1,5004,22000873,101136833,20.000000,Weekly
2,5004,22001170,100364230,37.500000,Monthly
3,5004,22001170,100366340,21.250000,Monthly
4,5004,22001236,100735436,12.090909,Weekly


In [0]:
#Appending the current data with the old order frequency data
order_freq_hist = spark.table(order_freq_data)
order_freq_hist=order_freq_hist.toPandas()
order_freq=order_freq_hist.append(order_freq)
order_freq.drop_duplicates(inplace= True)

In [0]:
order_cut=order_freq.drop_duplicates(subset=['Plant_ID','Material_No.','updated_VS_ID'])
order_cut.shape

Out[134]: (956, 5)

In [0]:
#order freqency data save
order_freq = spark.createDataFrame(order_cut)
order_freq.write.saveAsTable(order_freq_data,mode = 'overwrite')

In [0]:
# ves_smi_fin_freq[['Delivered_Quantity','Delivered_Quantity_UoM']] = ves_smi_fin_freq.Delivered_Quantity.str.split(' ',expand =True)
# ves_smi_fin_freq['Delivered_Quantity']=ves_smi_fin_freq['Delivered_Quantity'].astype(str)
# #removing comma from quantity variable
# d=list()
# for i in ves_smi_fin_freq['Delivered_Quantity']:
#   d.append(i.replace(',',''))
# ves_smi_fin_freq['Delivered_Quantity']=d
# ves_smi_fin_freq['Delivered_Quantity']=ves_smi_fin_freq['Delivered_Quantity'].astype(float)

# #Classification of UoM
# u=list()
# for i in ves_smi_fin_freq['Delivered_Quantity_UoM']:
#   if ((i=='FT') or (i=='IN')):
#     u.append('length')
#   elif ((i=='KG') or (i=='LB')):
#     u.append('mass')
#   elif i=='GAL':
#     u.append('volume')
#   else:
#     u.append('unit')
# ves_smi_fin_freq['Type_of_Measure']=u

# #Conversion kg to LB
# kg=2.205
# measure=list()
# U=list()
# for i,j in zip(ves_smi_fin_freq['Delivered_Quantity_UoM'],ves_smi_fin_freq['Delivered_Quantity']):
#   if i=='KG':
#     measure.append(j*2.205)
#     U.append('LB')
#   else:
#     measure.append(j)
#     U.append(i)

# ves_smi_fin_freq['Delivered_Quantity']=measure

In [0]:
#selecting the required fields and sorting the data in ascending order
ves_smi_fin_freq_cut=ves_smi_fin_freq[['Plant_ID','updated_VS_ID','Material_No.','Purchase_Order','PO_Item','PO_Create_Date','Purchase_Order_Scheduled_Qty','Delivery_date','Delivered_Quantity','Delivery_freq']]
ves_smi_fin_freq_cut.sort_values(by=['Plant_ID','Material_No.','updated_VS_ID','Delivery_date'], inplace = True)
ves_smi_fin_freq_uni=ves_smi_fin_freq[['Plant_ID','updated_VS_ID','Material_No.']].drop_duplicates()

In [0]:
#function to calculate moving average of delivered quantity
#if delivery is daily look at last 5 deliveries, if delivery is weekly look at last 4 deliveries, else look at last 3 deliveries
def mov_avg(p_id,vs_id,mat_id): 
  df1=ves_smi_fin_freq_cut[(ves_smi_fin_freq_cut['Plant_ID']==p_id)&(ves_smi_fin_freq_cut['updated_VS_ID']==vs_id)&(ves_smi_fin_freq_cut['Material_No.']==mat_id)]
  df1.reset_index()
  df1['MovingAverage'] = 0
  if (df1['Delivery_freq'].unique()== 'Daily'):
    if len(df1)==1:
      df1['MovingAverage']= df1['Delivered_Quantity']
    elif len(df1)==2:
      df1.iloc[0, df1.columns.get_loc('MovingAverage')]=df1.iloc[0]['Delivered_Quantity']
      df1.iloc[1, df1.columns.get_loc('MovingAverage')]=df1.iloc[0:2]['Delivered_Quantity'].mean()
    elif len(df1)==3:
      df1['MovingAverage'] = df1.rolling(window=3)['Delivered_Quantity'].mean()
      df1.iloc[0, df1.columns.get_loc('MovingAverage')]=df1.iloc[0]['Delivered_Quantity']
      df1.iloc[1, df1.columns.get_loc('MovingAverage')]=df1.iloc[0:2]['Delivered_Quantity'].mean()
    elif len(df1)==4:
      df1['MovingAverage'] = df1.rolling(window=4)['Delivered_Quantity'].mean()
      df1.iloc[0, df1.columns.get_loc('MovingAverage')]=df1.iloc[0]['Delivered_Quantity']
      df1.iloc[1, df1.columns.get_loc('MovingAverage')]=df1.iloc[0:2]['Delivered_Quantity'].mean() 
      df1.iloc[2, df1.columns.get_loc('MovingAverage')]=df1.iloc[0:3]['Delivered_Quantity'].mean()
    elif len(df1)>=5:
      df1['MovingAverage'] = df1.rolling(window=5)['Delivered_Quantity'].mean()
      df1.iloc[0, df1.columns.get_loc('MovingAverage')]=df1.iloc[0]['Delivered_Quantity']
      df1.iloc[1, df1.columns.get_loc('MovingAverage')]=df1.iloc[0:2]['Delivered_Quantity'].mean() 
      df1.iloc[2, df1.columns.get_loc('MovingAverage')]=df1.iloc[0:3]['Delivered_Quantity'].mean()
      df1.iloc[3, df1.columns.get_loc('MovingAverage')]=df1.iloc[0:4]['Delivered_Quantity'].mean()
    return df1
  elif df1['Delivery_freq'].unique()== 'Weekly':
    if len(df1)==1:
      df1['MovingAverage']= df1['Delivered_Quantity']
    elif len(df1)==2:
      df1.iloc[0, df1.columns.get_loc('MovingAverage')]=df1.iloc[0]['Delivered_Quantity']
      df1.iloc[1, df1.columns.get_loc('MovingAverage')]=df1.iloc[0:2]['Delivered_Quantity'].mean()
    elif len(df1)>2:
      df1['MovingAverage'] = df1.rolling(window=4)['Delivered_Quantity'].mean()
      df1.iloc[0, df1.columns.get_loc('MovingAverage')]=df1.iloc[0]['Delivered_Quantity']
      df1.iloc[1, df1.columns.get_loc('MovingAverage')]=df1.iloc[0:2]['Delivered_Quantity'].mean() 
      df1.iloc[2, df1.columns.get_loc('MovingAverage')]=df1.iloc[0:3]['Delivered_Quantity'].mean()
    return df1
  else:
    if len(df1)==1:
      df1['MovingAverage']= df1['Delivered_Quantity']
    elif len(df1)==2:
      df1.iloc[0, df1.columns.get_loc('MovingAverage')]=df1.iloc[0]['Delivered_Quantity']
      df1.iloc[1, df1.columns.get_loc('MovingAverage')]=df1.iloc[0:2]['Delivered_Quantity'].mean()
    elif len(df1)>2:
      df1['MovingAverage'] = df1.rolling(window=3)['Delivered_Quantity'].mean()
      df1.iloc[0, df1.columns.get_loc('MovingAverage')]=df1.iloc[0]['Delivered_Quantity']
      df1.iloc[1, df1.columns.get_loc('MovingAverage')]=df1.iloc[0:2]['Delivered_Quantity'].mean() 
    return df1  
avg_del_qty=pd.DataFrame()
for p_id,vs_id,mat_id in zip(ves_smi_fin_freq_uni['Plant_ID'],ves_smi_fin_freq_uni['updated_VS_ID'],ves_smi_fin_freq_uni['Material_No.']):
  temp_df= mov_avg(p_id,vs_id,mat_id)
  avg_del_qty=avg_del_qty.append(temp_df)
avg_del_qty.head(5)

Out[138]:

,Plant_ID,updated_VS_ID,Material_No.,Purchase_Order,PO_Item,PO_Create_Date,Purchase_Order_Scheduled_Qty,Delivery_date,Delivered_Quantity,Delivery_freq,MovingAverage
0,5004,100907802,22002012,4567777068,10,2022-01-04,19800.0,2021-11-19,19800.0,Yearly,19800.0
1,5004,100907802,22002012,4568036281,10,2022-01-04,5400.0,2021-12-21,5400.0,Yearly,12600.0
3,5004,100907802,22002012,4568323452,10,2022-01-04,21600.0,2022-01-25,21600.0,Yearly,15600.0
2,5004,100907802,22002012,4568305516,10,2022-01-04,18000.0,2022-02-08,18000.0,Yearly,15000.0
9372,5004,100907802,22002012,4568305516,10,2022-01-04,18000.0,2022-02-08,18000.0,Yearly,19200.0


In [0]:
#POqty to Avg delivery qty calculation
avg_del_qty['POqty_to_AvgDelQty'] = avg_del_qty['Purchase_Order_Scheduled_Qty']/avg_del_qty['MovingAverage']

In [0]:
# adding the moving average and ratio feature to the final table
moving_avg_table=avg_del_qty[['Plant_ID','Material_No.','updated_VS_ID','Purchase_Order','PO_Create_Date','PO_Item','MovingAverage','POqty_to_AvgDelQty']]
ves_smi_fin_freq_avg=pd.merge(ves_smi_fin_freq,moving_avg_table, how='left',on=['Plant_ID','Material_No.','updated_VS_ID','Purchase_Order','PO_Create_Date','PO_Item'])
ves_smi_fin_freq_avg.dropna(subset=['POqty_to_AvgDelQty'])
ves_smi_fin_freq_avg = ves_smi_fin_freq_avg[ves_smi_fin_freq_avg['POqty_to_AvgDelQty']!=np.inf]
ves_smi_fin_freq_avg.drop_duplicates(subset=ves_smi_fin_freq.columns,inplace=True)
print(ves_smi_fin_freq_avg.shape,ves_smi_fin_freq.shape)

(10097, 86) (10097, 84)

In [0]:
#Defining Distance Bin Information value function (Used as a feature in OTIF model)
class AttributeRelevance():
    def seq_palette(self, n_colors):
        return sns.cubehelix_palette(n_colors, start=.5, rot=-.75, reverse=True)

    def bulk_iv(self, feats, iv, woe_extremes=False):
        iv_dict = {}
        for f in feats:
            iv_df, iv_value = iv.calculate_iv(f)
            if woe_extremes:
                iv_dict[f.feature] = [iv_value, iv_df['woe'].min(), iv_df['woe'].max()]
                cols = ['iv', 'woe_min', 'woe_max']
            else:
                iv_dict[f.feature] = iv_value
                cols = ['iv']
        df = pd.DataFrame.from_dict(iv_dict, orient='index', columns=cols)
        return df

    def bulk_stats(self, feats, s):
        stats_dict = {}
        for f in feats:
            p_value, effect_size = s.calculate_chi(f)
            stats_dict[f.feature] = [p_value, effect_size]
        df = pd.DataFrame.from_dict(stats_dict, orient='index', columns=['p-value', 'effect_size'])
        return df

    def analyze(self, feats, iv, s=None, interpretation=False):
        df_iv = self.bulk_iv(feats, iv).sort_values(by='iv', ascending=False)
        if s is not None:
            df_stats = self.bulk_stats(feats, s)
            df_iv = df_iv.merge(df_stats, left_index=True, right_index=True)
        if interpretation:
            df_iv['iv_interpretation'] = df_iv['iv'].apply(iv.interpretation)
            if s is not None:
                df_iv['es_interpretation'] = df_iv['effect_size'].apply(s.interpretation)
        return df_iv

    def draw_iv(self, feats, iv):
        df = self.analyze(feats, iv)
        fig, ax = plt.subplots(figsize=(10, 6))
        sns.barplot(x=df.index, y='iv', data=df, palette=self.seq_palette(len(feats)))
        ax.set_title('IV values')
        plt.xticks(rotation=90)
        plt.show()

    def draw_woe_extremes(self, feats, iv):
        df = self.bulk_iv(feats, iv, woe_extremes=True).sort_values(by='iv', ascending=False)
        fig, ax = plt.subplots(figsize=(10, 6))
        sns.barplot(x=df.index, y='woe_min', data=df, palette=self.seq_palette(len(feats)))
        sns.barplot(x=df.index, y='woe_max', data=df, palette=self.seq_palette(len(feats)))
        ax.axhline(y=0, color='black', linewidth=1)
        ax.set_title('Range of WOE values')
        ax.set_ylabel('WOE')
        plt.xticks(rotation=90)
        plt.show()

    def draw_woe_multiplot(self, feats, iv):
        n = len(feats)
        nrows = int(np.ceil(n/3))
        fig, ax = plt.subplots(nrows=nrows, ncols=3, figsize=(15, nrows*4))
        for i in range(n):
            iv_df, iv_value = iv.calculate_iv(feats[i])
            sns.barplot(x=feats[i].feature, y='woe', data=iv_df, color='#455872', ax=fig.axes[i])

        for ax in fig.axes:
            plt.sca(ax)
            plt.xticks(rotation=50)

        plt.tight_layout()
        plt.show()

class Analysis():
    def seq_palette(self, n_colors):
        return sns.cubehelix_palette(n_colors, start=.5, rot=-.75, reverse=True)

    def group_by_feature(self, feat):
        df = feat.df_lite \
                            .groupby('bin') \
                            .agg({'label': ['count', 'sum']}) \
                            .reset_index()
        df.columns = [feat.feature, 'count', 'good']
        df['bad'] = df['count'] - df['good']
        return df

class StatsSignificance(Analysis):
    def calculate_chi(self, feat):
        df = self.group_by_feature(feat)
        df_chi = np.array(df[['good', 'bad']])
        n = df['count'].sum()

        chi = stats.chi2_contingency(df_chi)
        cramers_v = np.sqrt(chi[0] / n)          # assume that k=2 (good, bad)
        return chi[1], cramers_v

    @staticmethod
    def interpretation(cramers_v):
        if cramers_v < 0.1:
            return 'useless'
        elif cramers_v < 0.2:
            return 'weak'
        elif cramers_v < 0.4:
            return 'medium'
        elif cramers_v < 0.6:
            return 'strong'
        else:
            return 'very strong'

    def interpret_chi(self, feat):
        _, cramers_v = self.calculate_chi(feat)
        return self.interpretation(cramers_v)

    def print_chi(self, feat):
        p_value, cramers_v = self.calculate_chi(feat)
        print('P-value: %0.2f\nEffect size: %0.2f' % (p_value, cramers_v))
        print('%s is a %s predictor' % (feat.feature.capitalize(), self.interpretation(cramers_v)))


class IV(Analysis):
    @staticmethod
    def __perc_share(df, group_name):
        return df[group_name] / df[group_name].sum()

    def __calculate_perc_share(self, feat):
        df = self.group_by_feature(feat)
        df['perc_good'] = self.__perc_share(df, 'good')
        df['perc_bad'] = self.__perc_share(df, 'bad')
        df['perc_diff'] = df['perc_good'] - df['perc_bad']
        return df

    def __calculate_woe(self, feat):
        df = self.__calculate_perc_share(feat)
        df['woe'] = np.log(df['perc_good']/df['perc_bad'])
        df['woe'] = df['woe'].replace([np.inf, -np.inf], np.nan).fillna(0)
        return df

    def calculate_iv(self, feat):
        df = self.__calculate_woe(feat)
        df['iv'] = df['perc_diff'] * df['woe']
        return df, df['iv'].sum()

    def draw_woe(self, feat):
        iv_df, iv_value = self.calculate_iv(feat)
        fig, ax = plt.subplots(figsize=(10,6))
        sns.barplot(x=feat.feature, y='woe', data=iv_df, palette=self.seq_palette(len(iv_df.index)))
        ax.set_title('WOE visualization for: ' + feat.feature)
        plt.show()
        plt.show()

    @staticmethod
    def interpretation(iv):
        if iv < 0.02:
            return 'useless'
        elif iv < 0.1:
            return 'weak'
        elif iv < 0.3:
            return 'medium'
        elif iv < 0.5:
            return 'strong'
        else:
            return 'suspicious'

    def interpret_iv(self, feat):
        _, iv = self.calculate_iv(feat)
        return self.interpretation(iv)

    def print_iv(self, feat):
        _, iv = self.calculate_iv(feat)
        print('Information value: %0.2f' % iv)
        print('%s is a %s predictor' % (feat.feature.capitalize(), self.interpretation(iv)))

In [0]:
class CategoricalFeature():
    def __init__(self, df, feature):
        self.df = df
        self.feature = feature

    @property
    def df_lite(self):
        df_lite = self.df
        df_lite['bin'] = df_lite[self.feature].fillna('MISSING')
        return df_lite[['bin', 'label']]


class ContinuousFeature():
    def __init__(self, df, feature):
        self.df = df
        self.feature = feature
        self.bin_min_size = int(len(self.df) * 0.05)

    def __generate_bins(self, bins_num):
        df = self.df[[self.feature, 'label']]
        df['bin'] = pd.qcut(df[self.feature], bins_num, duplicates='drop') \
                    .apply(lambda x: x.left) \
                    .astype(float)
        return df

    def __generate_correct_bins(self, bins_max=10):
        for bins_num in range(bins_max, 1, -1):
            df = self.__generate_bins(bins_num)
            df_grouped = pd.DataFrame(df.groupby('bin') \
                                      .agg({self.feature: 'count',
                                            'label': 'sum'})) \
                                      .reset_index()
            r, p = stats.stats.spearmanr(df_grouped['bin'], df_grouped['label'])

            if (
#                     abs(r)==1 and                                                        # check if woe for bins are monotonic
                    df_grouped[self.feature].min() > self.bin_min_size                   # check if bin size is greater than 5%
                    and not (df_grouped[self.feature] == df_grouped['label']).any()      # check if number of good and bad is not equal to 0
            ):
                break

        return df

    @property
    def df_lite(self):
        df_lite = self.__generate_correct_bins()
        df_lite['bin'].fillna('MISSING', inplace=True)
        return df_lite[['bin', 'label']]

In [0]:
# defining distance bin creation by maximizing the information value
feat_distance = ContinuousFeature(ves_smi_fin_freq_avg, 'distance')
iv = IV()
ves_smi_fin_freq_avg['label']=ves_smi_fin_freq_avg['OTIF']
x=iv.group_by_feature(feat_distance)
bins=iv.group_by_feature(feat_distance)
iv_df, iv_value = iv.calculate_iv(feat_distance)
print(iv_df)
print('Information value: ', iv_value)

distance count good bad perc_good perc_bad perc_diff woe iv
0 -0.001 1834 0.0 1834.0 NaN 0.181638 NaN 0.0 NaN
1 82.543 834 0.0 834.0 NaN 0.082599 NaN 0.0 NaN
2 208.038 1373 0.0 1373.0 NaN 0.135981 NaN 0.0 NaN
3 409.879 1088 0.0 1088.0 NaN 0.107755 NaN 0.0 NaN
4 669.764 1304 0.0 1304.0 NaN 0.129147 NaN 0.0 NaN
5 1042.858 1164 0.0 1164.0 NaN 0.115282 NaN 0.0 NaN
6 1329.411 1274 0.0 1274.0 NaN 0.126176 NaN 0.0 NaN
7 2576.532 1226 0.0 1226.0 NaN 0.121422 NaN 0.0 NaN
Information value: 0.0

In [0]:
#creating the distance bin columns and then dummies for distance bins
k=list(x.distance)
k.append(ves_smi_fin_freq_avg['distance'].max())
b=[round(i,0) for i in k]
ves_smi_fin_freq_avg['range_distance'] = pd.cut(round(ves_smi_fin_freq_avg.distance,0),b, include_lowest=True)
print(ves_smi_fin_freq_avg['range_distance'].value_counts())
ves_smi_fin_freq_avg_dist=pd.get_dummies(ves_smi_fin_freq_avg,columns=['range_distance'])

(-0.001, 83.0] 1834
(208.0, 410.0] 1373
(670.0, 1043.0] 1304
(1329.0, 2577.0] 1274
(2577.0, 14958.0] 1226
(1043.0, 1329.0] 1164
(410.0, 670.0] 1088
(83.0, 208.0] 834
Name: range_distance, dtype: int64

In [0]:
#renaming the columns
#Depending on change in Vendor data with respect to distance, the distance bins can change
#Renaming the dynamically generated columns
col_rename_dict = {}
for col in ves_smi_fin_freq_avg_dist.columns.tolist():
  if 'range_distance_' in col:
    distance_bin_lower_limit = str(int(np.ceil(float(col.split('_distance_')[1].split(',')[0].split('(')[1]))))
    distance_bin_upper_limit = str(int(np.floor(float(col.split('_distance_')[1].split(',')[1].split(']')[0]))))
    new_col_name = 'range_bin_'+distance_bin_lower_limit+'_to_'+distance_bin_upper_limit
    col_rename_dict[col] = new_col_name

ves_smi_fin_freq_avg_dist.rename(columns=col_rename_dict,inplace=True)
ves_smi_fin_freq_avg_dist.drop('label',1,inplace=True)

In [0]:
#saving the model data as table
ves_smi_fin_freq_avg_dist=pd.get_dummies(ves_smi_fin_freq_avg_dist,columns=['Delivery_freq'])
col_names = ['_'.join(col.split(' ')) for col in ves_smi_fin_freq_avg_dist.columns.tolist()]
ves_smi_fin_freq_avg_dist.columns = col_names
model_data_hist = spark.table(model_data_final)
model_data_hist=model_data_hist.toPandas()
# ves_smi_fin_freq_avg_dist_tab = spark.createDataFrame(ves_smi_fin_freq_avg_dist)
# ves_smi_fin_freq_avg_dist_tab.write.saveAsTable(model_data_final,mode = 'overwrite')

In [0]:
ves_smi_fin_freq_avg_dist.rename(columns={'range_bin_0_to_83':'range_bin_7_to_70','range_bin_83_to_208':'range_bin_70_to_217','range_bin_208_to_410':'range_bin_217_to_326','range_bin_410_to_670':'range_bin_326_to_544','range_bin_670_to_1043':'range_bin_544_to_904','range_bin_1043_to_1329':'range_bin_904_to_1288','range_bin_1329_to_2577':'range_bin_1288_to_2153','range_bin_2577_to_14958':'range_bin_2153_to_2874'},inplace=True)
ves_smi_fin_freq_avg_dist['range_bin_2874_to_15463']=0
ves_smi_fin_freq_avg_dist['PO_Create_Month_5']=0

In [0]:
print(ves_smi_fin_freq_avg_dist.shape)
ves_smi_fin_freq_avg_dist.isnull().sum()

(69425, 103)
Out[154]: Plant_ID 0
Plant_Name 0
Vendor 0
VM_Name 0
Purchase_Order 0
PO_Item 0
Material_Group 0
Material_No. 0
Material_Description 0
First_GR_Date 0
Last_GR_Date 0
First_Statistical_delivery_Date 0
Delivery_date 0
Purchase_Order_Scheduled_Qty 0
Delivered_Quantity 0
Purchase_order_PO_item 0
Missed_PO 0
PO_Create_Date 0
updated_VS_country 0
updated_VS_ID 0
updated_VS_location 0
updated_VS_region 0
distance 0
vendor_percentage 0
vendor_material_cnt_per_plant 0
sku_type_single 0
dom_or_int_int 0
Material_group_type_R 0
nestle_managed_freight_yes 0
mode_of_transport_road 0
single_or_multisource_single 0
PO_Create_Month_1 0
PO_Create_Month_2 0
PO_Create_Month_3 0
PO_Create_Month_4 0
PO_Create_Month_5 0
PO_Create_Month_6 0
PO_Create_Month_7 0
PO_Create_Month_8 0
PO_Create_Month_9 0
PO_Create_Month_10 0
PO_Create_Month_11 0
PO_Create_Month_12 0
bus_day_count 0
lead_time 0
holiday_weekend_count 0
OTIF 0
l3m_date 0
l12m_date 0
no.of.complaint_3_months 0
no.of.complaint_12_months 0
no.of.complaint_3_months_major 0
no.of.complaint_3_months_minor 0
no.of.complaint_12_months_major 0
no.of.complaint_12_months_minor 0
no.of.complaint_3_months_open 0
no.of.complaint_3_months_open_major 0
no.of.complaint_3_months_open_minor 0
no.of.Complaint_type_Documentation_l12m 0
Complaint_type_Documentation_l12m_distinct 0
no.of.Complaint_type_Packaging_l12m 0
Complaint_type_Packaging_l12m_distinct 0
no.of.Complaint_type_Transportation_l12m 0
Complaint_type_Transportation_l12m_distinct 0
no.of.Complaint_type_Ingredient_l12m 0
Complaint_type_Ingredient_l12m_distinct 0
no.of.Complaint_type_Consumer_l12m 0
Complaint_type_Consumer_l12m_distinct 0
no.of.Complaint_type_Foreign_Material_l12m 0
Complaint_type_Foreign_Material_l12m_distinct 0
no.of.Complaint_type_Indigenous_Material_l12m 0
Complaint_type_Indigenous_Material_l12m_distinct 0
Risk_Level_High 0
Risk_Level_Low 0
Risk_Level_Medium 0
Rating_A 0
Rating_B 0
Rating_C 0
Rating_D 0
Rating_D_-_FND 0
Rating_F 0
Rating_Inactive 0
Rating_Out_of_Scope 0
days 0
Delivered_Quantity_UoM 10097
Type_of_Measure 10097
MovingAverage 0
POqty_to_AvgDelQty 0
range_bin_7_to_70 0
range_bin_70_to_217 0
range_bin_217_to_326 0
range_bin_326_to_544 0
range_bin_544_to_904 0
range_bin_904_to_1288 0
range_bin_1288_to_2153 0
range_bin_2153_to_2874 0
range_bin_2874_to_15463 0
Delivery_freq_Daily 0
Delivery_freq_Half_Yearly 0
Delivery_freq_Monthly 0
Delivery_freq_Quarterly 0
Delivery_freq_Weekly 0
Delivery_freq_Yearly 0
dtype: int64

In [0]:
#Appending the current data with the old model data
model_data_hist = spark.table(model_data_final)
model_data_hist=model_data_hist.toPandas()
model_data_hist.drop(columns=['Type_of_Measure','Delivered_Quantity_UoM'],inplace=True)
ves_smi_fin_freq_avg_dist=model_data_hist.append(ves_smi_fin_freq_avg_dist)
ves_smi_fin_freq_avg_dist.drop_duplicates(inplace= True)
ves_model_dt=spark.createDataFrame(ves_smi_fin_freq_avg_dist)
ves_model_dt.write.saveAsTable(model_data_final, mode = 'overwrite')

In [0]:
ves_smi_fin_freq_avg_dist.head(1)

Out[208]:

,Plant_ID,Plant_Name,Vendor,VM_Name,Purchase_Order,PO_Item,Material_Group,Material_No.,Material_Description,First_GR_Date,Last_GR_Date,First_Statistical_delivery_Date,Delivery_date,Purchase_Order_Scheduled_Qty,Delivered_Quantity,Purchase_order_PO_item,Missed_PO,PO_Create_Date,updated_VS_country,updated_VS_ID,updated_VS_location,updated_VS_region,distance,vendor_percentage,vendor_material_cnt_per_plant,sku_type_single,dom_or_int_int,Material_group_type_R,nestle_managed_freight_yes,mode_of_transport_road,single_or_multisource_single,PO_Create_Month_1,PO_Create_Month_2,PO_Create_Month_3,PO_Create_Month_4,PO_Create_Month_5,PO_Create_Month_6,PO_Create_Month_7,PO_Create_Month_8,PO_Create_Month_9,PO_Create_Month_10,PO_Create_Month_11,PO_Create_Month_12,bus_day_count,lead_time,holiday_weekend_count,OTIF,l3m_date,l12m_date,no.of.complaint_3_months,no.of.complaint_12_months,no.of.complaint_3_months_major,no.of.complaint_3_months_minor,no.of.complaint_12_months_major,no.of.complaint_12_months_minor,no.of.complaint_3_months_open,no.of.complaint_3_months_open_major,no.of.complaint_3_months_open_minor,no.of.Complaint_type_Documentation_l12m,Complaint_type_Documentation_l12m_distinct,no.of.Complaint_type_Packaging_l12m,Complaint_type_Packaging_l12m_distinct,no.of.Complaint_type_Transportation_l12m,Complaint_type_Transportation_l12m_distinct,no.of.Complaint_type_Ingredient_l12m,Complaint_type_Ingredient_l12m_distinct,no.of.Complaint_type_Consumer_l12m,Complaint_type_Consumer_l12m_distinct,no.of.Complaint_type_Foreign_Material_l12m,Complaint_type_Foreign_Material_l12m_distinct,no.of.Complaint_type_Indigenous_Material_l12m,Complaint_type_Indigenous_Material_l12m_distinct,Risk_Level_High,Risk_Level_Low,Risk_Level_Medium,Rating_A,Rating_B,Rating_C,Rating_D,Rating_D_-_FND,Rating_F,Rating_Inactive,Rating_Out_of_Scope,days,Delivered_Quantity_UoM,Type_of_Measure,MovingAverage,POqty_to_AvgDelQty,range_bin_7_to_70,range_bin_70_to_217,range_bin_217_to_326,range_bin_326_to_544,range_bin_544_to_904,range_bin_904_to_1288,range_bin_1288_to_2153,range_bin_2153_to_2874,range_bin_2874_to_15463,Delivery_freq_Daily,Delivery_freq_Half_Yearly,Delivery_freq_Monthly,Delivery_freq_Quarterly,Delivery_freq_Weekly,Delivery_freq_Yearly
0,5258,us pl medford,100364660,international paper company,4562214653,10,corrugated,43572559,blankcorrcircle23-23-23virginliner11.50,2020-01-23,2020-01-23,2020-01-22,2020-01-22,293760.0,293760.0,4562214653_10,No,2020-01-14,us,100855106,ramsey,mn,297.1187,97.51,2,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,6,8,2,1.0,2019-10-16,2019-01-14,15,15,2,13,2,13,0,0,0,0,0,15,1,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,7.183673,IEA,unit,293520.0,1.000818,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0


In [0]:
ves_smi_fin_freq_avg_dist_1=ves_smi_fin_freq_avg_dist[['Plant_ID','Plant_Name','updated_VS_ID','Purchase_Order','Material_No.','PO_Create_Date','Delivery_date','updated_VS_location','updated_VS_region','updated_VS_country']].copy()
ves_smi_fin_freq_avg_dist_1.drop_duplicates(inplace=True)
print(ves_smi_fin_freq_avg_dist_1.shape)
ves_model_dt1=spark.createDataFrame(ves_smi_fin_freq_avg_dist_1)
ves_model_dt1.write.saveAsTable('location_table', mode = 'overwrite')

(67203, 10)

In [0]:
/FileStore/tables/dec2021_mar2022.csv

In [0]:
#reading a new inventory file
invt = spark.read.csv("/FileStore/tables/dec2021_mar2022.csv", header="true", inferSchema="true")
invt = invt.drop('SnapshotDate_1')
#impporting the material in scope
mat_scope= spark.read.csv('/FileStore/tables/171Material_inScope.csv', header="true", inferSchema="true")
#converting into tuple
mat_tupple = mat_scope.select("Material").rdd.flatMap(lambda x: x).collect()

In [0]:
print((invt.count(), len(invt.columns)))

(5319562, 28)

In [0]:
#query to extract data for the materials in scope and saving the final table
invt.createOrReplaceTempView("my_table")
spark = SparkSession.builder.appName('invt').getOrCreate()
query = "select * from my_table where MaterialID IN {0}".format(tuple(mat_tupple))
# query = "select * from my_table where MaterialID  IN (22014017,22015453,22015528,43524805,43641219,43935306)"
invt_scope =spark.sql(query)
print((invt_scope.count(), len(invt_scope.columns)))

(141968, 28)

In [0]:
#appending the new inventory data with the old table
invt_scope=invt_scope.drop(invt_scope.SpecialStockNumber)
invt_hist=spark.table(historical_invt_raw_data)
print((invt_hist.count(), len(invt_hist.columns)))
invt_scope = invt_scope.union(invt_hist)
invt_scope=invt_scope.dropDuplicates()

# #Only inventory which is available on or before the Snapshot Date can be used
invt_scope=invt_scope.filter(invt_scope.SnapshotDate >= invt_scope.AvailableDate)
print((invt_scope.count(), len(invt_scope.columns)))
invt_scope.write.saveAsTable(historical_invt_raw_data,mode = 'overwrite')

(989808, 27)
(1127657, 27)

In [0]:
#importing MB51 data from flat file and selecting the required columns
MB51_data=pd.read_csv(MB51_flat_file)
MB51_cut=MB51_data[['Document Date - Key','Posting Date - Key','Plant - Key','Material - Key','Movement Type - Key','Qty in BUoM']]
MB51_cut.rename(columns={'Document Date - Key': 'Document_Date','Posting Date - Key':'Posting_Date',
                                 'Plant - Key':'Plant','Material - Key':'Material',
                                 'Movement Type - Key':'Movement_Type','Qty in BUoM':'Qty_in_BUoM',
                                },inplace=True)

In [0]:
#selcting Goods issue specific movement type data and calculating the consumption
MB51_cut_1=MB51_cut[(invt_cut['Movement_Type']=='261')|(invt_cut['Movement_Type']=='711')|(invt_cut['Movement_Type']=='ZU1')|(invt_cut['Movement_Type']=='Z11')|(invt_cut['Movement_Type']=='201')|(invt_cut['Movement_Type']=='221')|(invt_cut['Movement_Type']=='717')|(invt_cut['Movement_Type']=='262')|(invt_cut['Movement_Type']=='202')|(invt_cut['Movement_Type']=='ZU2')]

MB51_cut_1=MB51_cut_1.sort_values(by=['Plant','Material','Posting_Date'])
invt_consumption=pd.pivot_table(MB51_cut_1, values = 'Qty_in_BUoM', index=['Plant','Material','Posting_Date'],aggfunc='sum').reset_index()

In [0]:
#importing data from the saved consumption table
consumption_hist=spark.table(inventory_consumption_mb51)
consumption_hist=consumption_hist.toPandas()
invt_consumption=invt_consumption.append(consumption_hist,ignore_index= True)
invt_consumption.drop_duplicates(inplace=True)

In [0]:
invt_consumption=spark.createDataFrame(invt_consumption)
invt_consumption.write.saveAsTable(inventory_consumption_mb51, mode = 'overwrite')